<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/mcda_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCDA: PROJECT - Final Algorithm

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np

class DataAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.pesos = None
        self.tipos_dados = None
        self.proporcionalidade = None
        self.valores_bom = None
        self.valores_neutro = None
        self.calculo_df = None
        # Adicione o mapeamento de string aqui para ser acessível
        self.string_columns_map = {}


    def load_and_prepare_data(self):
        """Carrega e prepara os dados de forma genérica para strings"""
        self.df = pd.read_csv(self.file_path, encoding='UTF-8', sep=',', skipinitialspace=True)

        # Remove colunas totalmente vazias
        self.df = self.df.dropna(axis=1, how='all')

        # Extrai configurações ANTES de mapear colunas string
        self.pesos = self.df.iloc[0].dropna().astype(float)
        self.tipos_dados = self.df.iloc[1].dropna()
        self.proporcionalidade = self.df.iloc[2].dropna()

        # Identifica automaticamente colunas string e seus pts_string
        # É importante fazer isso *depois* de ler os tipos_dados
        self._mapear_colunas_string()

        # Extrai BOM/NEUTRO
        # Usar .get para evitar KeyError se 'Modelo' não for a primeira coluna
        bom_row = self.df[self.df.iloc[:, 0] == 'BOM']
        neutro_row = self.df[self.df.iloc[:, 0] == 'NEUTRO']

        if not bom_row.empty:
             self.valores_bom = bom_row.iloc[0].dropna()
        else:
             print("Aviso: Linha 'BOM' não encontrada.")
             self.valores_bom = pd.Series(dtype='object')

        if not neutro_row.empty:
             self.valores_neutro = neutro_row.iloc[0].dropna()
        else:
             print("Aviso: Linha 'NEUTRO' não encontrada.")
             self.valores_neutro = pd.Series(dtype='object')


        # Cria calculo_df pulando as 5 primeiras linhas (pesos, tipo, prop, BOM, NEUTRO)
        # Assume que 'Modelo' é a primeira coluna para a lógica BOM/NEUTRO
        self.calculo_df = self.df.iloc[5:].reset_index(drop=True)
        self._convert_data_types()

    def _convert_data_types(self):
        """Converte os tipos de dados conforme especificado"""
        # Garante que valores_bom e valores_neutro sejam Series
        if self.valores_bom is None: self.valores_bom = pd.Series(dtype='object')
        if self.valores_neutro is None: self.valores_neutro = pd.Series(dtype='object')

        for col in self.tipos_dados.index:
            if col not in self.calculo_df.columns:
                print(f"Aviso: Coluna '{col}' definida nos tipos mas não encontrada nos dados para cálculo.")
                continue

            tipo = self.tipos_dados.get(col) # Usar .get para segurança

            if tipo == 'number':
                # Tenta converter BOM/NEUTRO para float, usa NaN se falhar
                self.valores_bom[col] = pd.to_numeric(self.valores_bom.get(col), errors='coerce')
                self.valores_neutro[col] = pd.to_numeric(self.valores_neutro.get(col), errors='coerce')
                # Converte coluna de dados
                self.calculo_df[col] = pd.to_numeric(self.calculo_df[col], errors='coerce')

            elif tipo == 'boolean':
                 # --- CORREÇÃO BOOLEAN ---
                 # Converte BOM/NEUTRO: 'TRUE' -> True, outros -> False
                 self.valores_bom[col] = str(self.valores_bom.get(col, '')).strip().upper() == 'TRUE'
                 self.valores_neutro[col] = str(self.valores_neutro.get(col, '')).strip().upper() == 'TRUE'
                 # Converte coluna de dados com mapeamento robusto
                 self.calculo_df[col] = self.calculo_df[col].astype(str).str.strip().str.upper().map({'TRUE': True, 'FALSE': False})
                 # Define um padrão para valores não mapeados (ex: NaN ou outros textos)
                 self.calculo_df[col] = self.calculo_df[col].fillna(False).astype(bool) # Trata NaN e força tipo bool

            elif tipo == 'string':
                # Apenas garante que a coluna de dados seja string
                self.calculo_df[col] = self.calculo_df[col].astype(str)

            # Lidar com colunas pts_string (não precisam de conversão aqui)
            elif tipo == 'pts_string':
                 continue # Ignora pts_string na conversão principal

            else:
                 print(f"Aviso: Tipo de dado desconhecido '{tipo}' para coluna '{col}'.")


    def calcular_pontuacao_numerica(self, valor, bom, neutro, peso, proporcionalidade_tipo):
        """Calcula pontuação para dados numéricos e booleanos"""

        # --- CORREÇÃO BOOLEAN LOGIC ---
        # Trata explicitamente valores booleanos primeiro
        # Usar isinstance com np.bool_ também por causa do Pandas/Numpy
        if isinstance(valor, (bool, np.bool_)):
            # Caso especial: BOM e NEUTRO são iguais (ambos True ou ambos False)
            if bom == neutro:
                return peso if valor == bom else 0 # Pontua se igual ao único alvo, senão 0
            # Caso normal: BOM e NEUTRO são diferentes
            else:
                if valor == bom:
                    return peso  # Acertou o BOM
                elif valor == neutro:
                    return 0     # Acertou o NEUTRO
                else:
                    # Não é nem BOM nem NEUTRO (deve ser o oposto do BOM e do NEUTRO)
                    return -peso # Penalidade

        # Se não for booleano, segue para a lógica numérica
        # Verifica se algum valor essencial é NaN antes de prosseguir
        if pd.isna(valor) or pd.isna(bom) or pd.isna(neutro) or pd.isna(peso):
             return 0 # Não pontua se faltar informação

        # Se BOM == NEUTRO para números, retorna 0 (ou peso se valor for igual?)
        # Mantendo a lógica original: retorna 0 se bom == neutro
        if bom == neutro:
            return 0

        # Tenta converter para float (já deveriam ser, mas por segurança)
        try:
            valor = float(valor)
            bom = float(bom)
            neutro = float(neutro)
            peso = float(peso)
        except (ValueError, TypeError):
            # Se a conversão falhar aqui, algo está errado antes
            print(f"Aviso: Falha na conversão para float em calcular_pontuacao_numerica - Valor: {valor}, Bom: {bom}, Neutro: {neutro}")
            return 0

        # Lógica de proporcionalidade (mantida como estava, mas agora com valores mais seguros)
        # Adicionando tratamento para divisão por zero ou neutro zero
        if proporcionalidade_tipo == 'proportional':
            if bom > neutro: # Caso normal crescente
                 if valor >= bom:
                     return peso
                 elif valor >= neutro:
                      # Evita divisão por zero se bom == neutro (já tratado acima)
                     return ((valor - neutro) / (bom - neutro)) * peso
                 else: # Abaixo do neutro
                     # Evita divisão por zero se neutro for 0 e valor negativo
                     denominador_penalidade = neutro if neutro != 0 else 1 # Usa 1 se neutro for 0
                     return -((abs(neutro - valor)) / abs(denominador_penalidade)) * peso # Penalidade proporcional à distância do neutro
            elif bom < neutro: # Caso decrescente (ex: menor é melhor até BOM) - Incomum para 'proportional' mas possível
                 if valor <= bom:
                     return peso
                 elif valor <= neutro:
                     return ((valor - bom) / (neutro - bom)) * peso # Inverte a lógica
                 else: # Acima do neutro (pior)
                    denominador_penalidade = neutro if neutro != 0 else 1
                    return -((abs(valor - neutro)) / abs(denominador_penalidade)) * peso
            else: # bom == neutro
                 return 0

        elif proporcionalidade_tipo == 'i_proportional':
             if bom < neutro: # Caso normal decrescente (menor é melhor)
                 if valor <= bom:
                     return peso
                 elif valor <= neutro:
                      # Evita divisão por zero se neutro == bom (já tratado acima)
                     return ((neutro - valor) / (neutro - bom)) * peso
                 else: # Acima do neutro (pior)
                     denominador_penalidade = neutro if neutro != 0 else 1
                     return -((abs(valor - neutro)) / abs(denominador_penalidade)) * peso # Penalidade proporcional
             elif bom > neutro: # Caso crescente (maior é melhor até BOM) - Incomum para 'i_proportional'
                 if valor >= bom:
                     return peso
                 elif valor >= neutro:
                     return ((bom - valor) / (bom - neutro)) * peso # Inverte a lógica
                 else: # Abaixo do neutro (pior)
                     denominador_penalidade = neutro if neutro != 0 else 1
                     return -((abs(neutro - valor)) / abs(denominador_penalidade)) * peso
             else: # bom == neutro
                 return 0

        return 0 # Caso tipo de proporcionalidade não seja reconhecido


    def _mapear_colunas_string(self):
        """Mapeia automaticamente colunas string para suas colunas pts_string"""
        self.string_columns_map = {}
        if self.tipos_dados is None: # Garante que tipos_dados foi carregado
             print("Erro: tipos_dados não carregado antes de mapear colunas string.")
             return

        string_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados[col] == 'string']

        df_cols = list(self.df.columns) # Lista das colunas originais

        for col in string_cols:
            try:
                # Encontra a posição da coluna string
                col_idx = df_cols.index(col)

                # A coluna pts_string deve ser a próxima coluna
                if col_idx + 1 < len(df_cols):
                    pts_potential_col = df_cols[col_idx + 1]
                    # Verifica se a próxima coluna é realmente a de pontos (opcional, mas bom)
                    # Poderia verificar se o tipo dela é 'pts_string' na linha 1, mas vamos confiar na posição por enquanto
                    new_pts_col_name = f"{col},pts_string"
                    self.string_columns_map[col] = new_pts_col_name # Armazena o nome *novo*

                    # Renomeia a coluna no DataFrame original
                    self.df = self.df.rename(columns={pts_potential_col: new_pts_col_name})
                    print(f"Coluna string '{col}' mapeada para '{new_pts_col_name}' (original: '{pts_potential_col}')")

                else:
                    print(f"Aviso: Coluna string '{col}' não tem uma coluna seguinte para ser pts_string.")
            except ValueError:
                 print(f"Aviso: Coluna string '{col}' definida nos tipos não encontrada no CSV.")
            except Exception as e:
                 print(f"Erro ao mapear coluna string '{col}': {e}")


    def calcular_pontuacao_string(self, valor, coluna):
        """Calcula pontuação genérica para qualquer string"""
        # Obtém o nome CORRETO da coluna de pontos, que foi renomeada
        pts_col = self.string_columns_map.get(coluna)

        if not pts_col:
            # Isso não deveria acontecer se _mapear_colunas_string funcionou
            print(f"Aviso interno: Mapeamento para coluna string '{coluna}' não encontrado.")
            return 0

        if pts_col not in self.df.columns:
             # Isso pode acontecer se o renomear falhou ou a coluna não existia
             print(f"Aviso: Coluna de pontos '{pts_col}' esperada para '{coluna}' não existe no DataFrame.")
             return 0

        try:
            # --- CORREÇÃO STRING ---
            # 1. Criar o mapeamento a partir das linhas de dados do df original
            dados_mapa = self.df.iloc[5:].copy()

            # 2. Converter a coluna de pontos para numérico DEPOIS de selecionar os dados
            dados_mapa[pts_col] = pd.to_numeric(dados_mapa[pts_col], errors='coerce')

            # 3. Remover linhas onde a string original OU o ponto convertido são NaN
            dados_mapa = dados_mapa.dropna(subset=[coluna, pts_col])

            # 4. Criar o dicionário de mapeamento (String -> Ponto Numérico)
            mapping = pd.Series(dados_mapa[pts_col].values, index=dados_mapa[coluna]).to_dict()

            # Se o mapping estiver vazio após a limpeza, retorna 0
            if not mapping:
                 print(f"Aviso: Mapeamento de pontuação para '{coluna}' está vazio após limpeza.")
                 return 0

            # Remove espaços e busca o valor no mapeamento
            valor_clean = str(valor).strip()
            # O valor buscado no dicionário JÁ É float (ou int)
            pontuacao_base = mapping.get(valor_clean, 0) # Pega o ponto base (já numérico)

            # Multiplica o ponto base pelo peso da coluna string
            peso_coluna = float(self.pesos.get(coluna, 0)) # Pega o peso com segurança
            resultado = pontuacao_base * peso_coluna

            # Debugging extra (opcional):
            # print(f"  Debug String: Col: {coluna}, Valor: '{valor_clean}', Ponto Base: {pontuacao_base}, Peso: {peso_coluna}, Result: {resultado}")

            return resultado

        except Exception as e:
            # Captura erros mais específicos se possível
            print(f"Erro detalhado ao calcular pontuação para '{coluna}' (Valor: '{valor}'): {type(e).__name__} - {str(e)}")
            # Imprimir o estado do mapeamento pode ajudar
            # print(f"  Debug - Estado do mapping problemático: {mapping}")
            return 0

    def calcular_pontuacoes(self):
        """Calcula todas as pontuações com verificação extra"""
        if self.calculo_df is None or self.tipos_dados is None or self.pesos is None:
             print("Erro: Dados não carregados ou preparados corretamente. Execute load_and_prepare_data() primeiro.")
             return

        print("Colunas disponíveis para cálculo:", list(self.calculo_df.columns))
        print("Pesos aplicáveis:", self.pesos.to_dict())
        print("Tipos de Dados:", self.tipos_dados.to_dict())


        for col in self.pesos.index:
            # Verifica se a coluna de peso existe nos dados de cálculo
            if col not in self.calculo_df.columns:
                # Verifica se é uma coluna string (cujo cálculo usa a coluna original)
                if col not in self.string_columns_map:
                     print(f"  Aviso: Coluna de peso '{col}' não encontrada no DataFrame de cálculo e não é uma coluna string mapeada.")
                     continue
                # Se for string, o cálculo usará a coluna original, então está OK não estar em calculo_df diretamente,
                # mas precisamos garantir que o tipo é 'string'
                elif self.tipos_dados.get(col) != 'string':
                     print(f"  Aviso: Coluna de peso '{col}' não encontrada no DataFrame de cálculo, mas mapeada como string ({self.string_columns_map.get(col)}). Verifique a consistência.")
                     continue
                # Se for string mapeada, o processamento ocorrerá no bloco 'string' abaixo

            print(f"\nProcessando coluna: {col}")

            # Pega informações com segurança usando .get()
            tipo = self.tipos_dados.get(col)
            proporcionalidade = self.proporcionalidade.get(col)
            peso = self.pesos.get(col, 0) # Usa 0 se peso não for encontrado por algum motivo
            valor_bom = self.valores_bom.get(col)
            valor_neutro = self.valores_neutro.get(col)
            pont_col_name = f"{col}_pontuacao" # Nome da coluna de resultado

            try:
                if tipo in ['number', 'boolean']:
                    # Verifica se todas as informações necessárias estão presentes
                    if pd.isna(valor_bom) or pd.isna(valor_neutro):
                         print(f"  Aviso: Valores BOM ({valor_bom}) ou NEUTRO ({valor_neutro}) ausentes/inválidos para '{col}'. Pulando pontuação.")
                         self.calculo_df[pont_col_name] = 0
                         continue
                    if not proporcionalidade:
                         print(f"  Aviso: Proporcionalidade não definida para '{col}'. Pulando pontuação.")
                         self.calculo_df[pont_col_name] = 0
                         continue

                    print(f"  Tipo: {tipo}, Proporcionalidade: {proporcionalidade}")
                    print(f"  Valores BOM: {valor_bom}, NEUTRO: {valor_neutro}, Peso: {peso}")

                    self.calculo_df[pont_col_name] = self.calculo_df.apply(
                        lambda row: self.calcular_pontuacao_numerica(
                            row[col],
                            valor_bom,
                            valor_neutro,
                            peso,
                            proporcionalidade
                        ) if col in row else 0, # Adiciona verificação se col existe na linha
                        axis=1
                    )

                elif tipo == 'string':
                    # Verifica se a coluna original existe
                    if col not in self.calculo_df.columns:
                         print(f"  Erro Interno: Coluna string '{col}' não encontrada em calculo_df para aplicar pontuação.")
                         self.calculo_df[pont_col_name] = 0
                         continue

                    pts_col_esperado = self.string_columns_map.get(col)
                    print(f"  Tipo: {tipo}. Coluna de pontos esperada: {pts_col_esperado}. Peso: {peso}")
                    # A validação da existência de pts_col é feita dentro de calcular_pontuacao_string

                    self.calculo_df[pont_col_name] = self.calculo_df.apply(
                        lambda row: self.calcular_pontuacao_string(row[col], col),
                        axis=1
                    )

                # Imprime resultado parcial se a coluna foi criada
                if pont_col_name in self.calculo_df:
                    print(f"  Resultado parcial (Top 5):\n{self.calculo_df[[col, pont_col_name]].head().to_string(index=False)}")
                else:
                     print(f"  Aviso: Coluna de pontuação '{pont_col_name}' não foi criada.")

            except KeyError as e:
                 print(f"  Erro de Chave (KeyError) ao processar coluna '{col}': {e}. Verifique se a coluna existe e os nomes estão corretos nas configurações e no CSV.")
                 self.calculo_df[pont_col_name] = 0
            except Exception as e:
                print(f"  Erro inesperado ao processar coluna '{col}': {type(e).__name__} - {str(e)}")
                import traceback
                # traceback.print_exc() # Descomente para obter traceback completo se necessário
                self.calculo_df[pont_col_name] = 0

        # Calcula pontuação total
        colunas_pontuacao = [c for c in self.calculo_df.columns if c.endswith('_pontuacao')]
        if not colunas_pontuacao:
             print("\nNenhuma coluna de pontuação foi gerada. Pontuação total não calculada.")
             self.calculo_df['Pontuacao_Total'] = 0
        else:
             print(f"\nColunas usadas para Pontuação Total: {colunas_pontuacao}")
             self.calculo_df['Pontuacao_Total'] = self.calculo_df[colunas_pontuacao].sum(axis=1)

        print("\nResumo final das pontuações:")
        # Seleciona colunas de forma segura
        cols_to_show = [self.calculo_df.columns[0]] + colunas_pontuacao + ['Pontuacao_Total']
        valid_cols_to_show = [c for c in cols_to_show if c in self.calculo_df.columns]
        print(self.calculo_df[valid_cols_to_show].head().to_string(index=False))


  # Mostra as pontuações em forma de código (debug apenas)
    def mostrar_pontuacoes(self, apenas_pontuacoes=True):
      """Exibe o DataFrame com as pontuações calculadas

      Args:
          apenas_pontuacoes (bool): Se True, mostra apenas colunas de pontuação
                                    Se False, mostra todas as colunas
      """
      if self.calculo_df is None:
           print("DataFrame de cálculo não disponível.")
           return

      try:
           # Tenta importar display dinamicamente se estiver em um notebook
           from IPython.display import display
      except ImportError:
           # Se não estiver em um notebook, usa print
           display = print

      if apenas_pontuacoes:
          # Filtra apenas colunas que terminam com '_pontuacao' ou são 'Modelo' ou 'Pontuacao_Total'
          # Assume que a primeira coluna é o identificador
          modelo_col = self.calculo_df.columns[0]
          cols_pontuacao = [c for c in self.calculo_df.columns if c.endswith('_pontuacao')]
          cols_total = ['Pontuacao_Total'] if 'Pontuacao_Total' in self.calculo_df.columns else []
          cols = [modelo_col] + cols_pontuacao + cols_total
          valid_cols = [c for c in cols if c in self.calculo_df.columns]
          display(self.calculo_df[valid_cols])
      else:
          display(self.calculo_df)

    def gerar_graficos(self):
        """Gera os gráficos de análise"""
        print("\n--- Gerando Gráficos ---")
        if self.calculo_df is None:
            print("DataFrame de cálculo não disponível para gerar gráficos.")
            return
        try:
            self._gerar_grafico_barras()  # Gráfico de pontuação total
            self._gerar_grafico_numericos()  # Gráficos para cada coluna numérica
            self._gerar_grafico_strings()
            self._gerar_grafico_booleanos()
            print("--- Gráficos Gerados ---")
        except Exception as e:
            print(f"Erro ao gerar gráficos: {e}")
            import traceback
            traceback.print_exc()

    def _gerar_grafico_numericos(self):
        """Gráfico de barras para colunas numéricas com verificação de proporcionalidade"""
        if self.tipos_dados is None:
            print("Tipos de dados não carregados, pulando gráfico de numéricos.")
            return

        # Identifica colunas numéricas
        numeric_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados[col] == 'number' and col in self.calculo_df.columns]

        if not numeric_cols:
            print("Nenhuma coluna numérica encontrada para gerar gráficos.")
            return

        modelo_col = self.calculo_df.columns[0]  # Assume primeira coluna como identificador

        for col_num in numeric_cols:
            print(f"Gerando gráfico para coluna numérica: '{col_num}'")

            # Prepara dados
            df_num = self.calculo_df[[modelo_col, col_num]].copy()
            df_num = df_num[~df_num[modelo_col].isin(['BOM', 'NEUTRO'])]
            df_num.dropna(subset=[col_num], inplace=True)

            if df_num.empty:
                print(f"Nenhum dado válido para plotar no gráfico de '{col_num}'.")
                continue

            # Obtém valores de referência e tipo de proporcionalidade
            valor_bom = self.valores_bom.get(col_num)
            valor_neutro = self.valores_neutro.get(col_num)
            proporcionalidade = self.proporcionalidade.get(col_num, '')

            # Verifica se é inversamente proporcional
            inversamente_proporcional = proporcionalidade.lower() == 'i_proportional'

            # Adiciona informação ao título
            titulo = f"{col_num} {'(Inversamente Proporcional)' if inversamente_proporcional else ''}"

            # Classifica como positivo/negativo considerando a proporcionalidade
            df_num['Color_Category'] = df_num.apply(
                lambda x: self._classificar_valor(x[col_num], valor_bom, valor_neutro, inversamente_proporcional),
                axis=1
            )

            # Define o mapa de cores
            color_map = {
                'Positiva': 'lightgreen',
                'Negativa': 'palevioletred',
                'Neutra': 'lightgray'
            }

            # Ordena por valor (maior no topo)
            df_num_sorted = df_num.sort_values(col_num, ascending=not inversamente_proporcional)

            # Cria gráfico
            fig = px.bar(df_num_sorted,
                        x=col_num,
                        y=modelo_col,
                        orientation='h',
                        color='Color_Category',
                        color_discrete_map=color_map,
                        hover_name=modelo_col,
                        hover_data={col_num: ':.2f'},
                        title=titulo)

            # Configurações de layout
            fig.update_layout(
                yaxis={'categoryorder': 'total ascending'},
                plot_bgcolor='white',
                paper_bgcolor='white',
                legend_title_text='Resultado',
                xaxis_title=col_num,
                yaxis_title='Modelo'
            )

            # Adiciona linhas de referência e anotações
            if pd.notna(valor_neutro):
                self._adicionar_referencia(fig, float(valor_neutro), len(df_num_sorted),
                                        "Mínimo Aceitável<br>(NEUTRO)", inversamente_proporcional)

            if pd.notna(valor_bom):
                self._adicionar_referencia(fig, float(valor_bom), len(df_num_sorted),
                                        "Desejável (BOM)", inversamente_proporcional)

            print(f"  Gráfico de {col_num} pronto para exibição.")
            fig.show()

    def _classificar_valor(self, valor, valor_bom, valor_neutro, inversamente_proporcional):
        """Classifica o valor como Positiva, Negativa ou Neutra"""
        if pd.isna(valor_neutro):
            return 'Neutra'

        valor = float(valor)
        valor_neutro = float(valor_neutro)

        if inversamente_proporcional:
            return 'Positiva' if valor <= valor_neutro else 'Negativa'
        else:
            return 'Positiva' if valor >= valor_neutro else 'Negativa'

    def _adicionar_referencia(self, fig, valor, y_pos, texto, inversamente_proporcional):
        """Adiciona linha de referência e anotação ao gráfico"""
        fig.add_vline(x=valor, line_dash='dash', line_color='black')

        fig.add_annotation(
            x=valor,
            y=y_pos - 1,
            xref="x",
            yref="y",
            text=texto,
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

    def _gerar_grafico_barras(self):
        """Gráfico de barras para pontuação total com linha ideal fixa e indicação do melhor produto."""
        print("Gerando gráfico de barras...")
        # Garante que colunas e pesos existem
        if 'Pontuacao_Total' not in self.calculo_df.columns:
            print("Coluna 'Pontuacao_Total' não encontrada. Pulando gráfico de barras.")
            return
        # Verifica se os pesos foram carregados para calcular o máximo teórico
        if self.pesos is None or self.pesos.empty:
             print("Pesos não definidos. Não é possível calcular pontuação máxima teórica. Usando máximo atingido.")
             # Como fallback, podemos usar o máximo atingido ou um valor fixo como 1.0
             # Vamos definir como 1.0 se os pesos não estiverem disponíveis.
             max_score_teorico = 1.0
        else:
            # Calcula a pontuação máxima teórica (soma dos pesos)
            max_score_teorico = self.pesos.sum()
            print(f"  Pontuação máxima teórica calculada (soma dos pesos): {max_score_teorico:.2f}")

        modelo_col = self.calculo_df.columns[0] # Assume primeira coluna como identificador

        df_analise = self.calculo_df[[modelo_col, 'Pontuacao_Total']].copy()
        # Remove linhas BOM/NEUTRO
        df_analise = df_analise[~df_analise[modelo_col].isin(['BOM', 'NEUTRO'])]

        # Remove linhas com pontuação NaN antes de prosseguir e ordenar
        df_analise.dropna(subset=['Pontuacao_Total'], inplace=True)

        if df_analise.empty:
             print("Nenhum dado válido para plotar no gráfico de barras após remover BOM/NEUTRO/NaN.")
             return

        df_analise['Color_Category'] = df_analise.apply(
            lambda x: ('Positiva' if x['Pontuacao_Total'] >= 0 else 'Negativa'),
            axis=1
        )

        color_map = {
            'Positiva': 'lightgreen',
            'Negativa': 'palevioletred'
        }

        # Ordena por pontuação DESCENDENTE para plotagem e identificação do melhor
        df_analise_sorted = df_analise.sort_values('Pontuacao_Total', ascending=False)

        # Identifica o melhor produto (primeira linha após ordenar)
        # Adiciona verificação caso df_analise_sorted fique vazio após dropna
        if df_analise_sorted.empty:
             print("Nenhum produto restante após limpeza para identificar o melhor.")
             top_product_name = "N/A"
             top_product_score = pd.NA
        else:
             top_product = df_analise_sorted.iloc[0]
             top_product_name = top_product[modelo_col]
             top_product_score = top_product['Pontuacao_Total']
             print(f"  Melhor produto identificado: {top_product_name} (Score: {top_product_score:.2f})")


        fig = px.bar(df_analise_sorted,
                    x='Pontuacao_Total',
                    y=modelo_col, # Modelo no eixo Y
                    orientation='h',
                    color='Color_Category',
                    color_discrete_map=color_map,
                    hover_name=modelo_col,
                    hover_data={'Pontuacao_Total':':.2f'}, # Formata hover
                    title='Pontuação Total dos Modelos')

        # Atualiza layout - Ordena eixo Y pela pontuação (melhor em cima)
        fig.update_layout(
            yaxis={'categoryorder': 'total ascending'},
            plot_bgcolor='white',
            paper_bgcolor='white',
            legend_title_text='Resultado',
            xaxis_title='Pontuação Total',
            yaxis_title='Modelo'
        )

        # Linha de referência no neutro
        fig.add_vline(x=0, line_dash='dash', line_color='black')

        # Linha de referência no desejável
        fig.add_vline(x=max_score_teorico, line_dash='dash', line_color='black')

        # Linha Desejável (BOM)
        fig.add_annotation(
            x=max_score_teorico,
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Desejável (BOM)<br>(NEUTRO)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

        # Linha Mínimo Aceitável (NEUTRO)
        fig.add_annotation(
            x=0,
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Mínimo Aceitável<br>(NEUTRO)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

        # --- Anotação para Melhor Produto ---
        # Só adiciona a anotação se identificamos um melhor produto válido
        if top_product_name != "N/A" and pd.notna(top_product_score):
            fig.add_annotation(
                    x=top_product_score,    # Posição X na pontuação do melhor
                    y=top_product_name,     # Posição Y no nome do melhor (categoria no eixo y)
                    text="🏆 Melhor Produto", # O texto da anotação
                    showarrow=False,        # SEM seta, como você definiu
                    font=dict(
                        color="#00008B",    # Cor da fonte (Azul Escuro)
                        size=12
                    ),
                    # align="center",         # Remova ou comente esta linha

                    # --- Adições para Fundo e Borda ---
                    bgcolor="white",        # Cor do fundo = branco
                    bordercolor="black",    # Cor da borda = preta
                    borderwidth=1,        # Largura da borda = 1 pixel
                    borderpad=4,          # Espaçamento interno = 4 pixels
                    # --- Ajuste de Posição (sem seta) ---
                    # Usamos xshift/yshift para mover o *texto* em relação ao ponto (x,y)
                    # Ajuste esses valores para posicionar onde desejar (ex: acima e à direita da ponta da barra)
                    xshift=-15,            # Experimente com valores negativos (ajuste conforme necessário)
                    yshift=25
            )

        # As anotações antigas que você colou não são necessárias pois usamos annotation_text nas vlines
        # e adicionamos uma anotação específica para o melhor produto.

        print("  Gráfico de barras pronto para exibição.")
        fig.show()


    def _gerar_grafico_strings(self):
        """Gráfico melhorado para variáveis de string - agora para todas as colunas string"""
        if self.tipos_dados is None:
            print("Tipos de dados não carregados, pulando gráfico de strings.")
            return

        string_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados.get(col) == 'string' and col in self.calculo_df.columns]

        if not string_cols:
            return

        modelo_col = self.calculo_df.columns[0] # Primeira coluna como identificador

        for col_string in string_cols:
            print(f"Gerando gráfico para string: '{col_string}'")
            pts_col_name = self.string_columns_map.get(col_string)

            if not pts_col_name or pts_col_name not in self.df.columns:
                print(f"Aviso: Coluna de pontos '{pts_col_name}' para '{col_string}' não encontrada. Pulando gráfico.")
                continue

            try:
                # Cria mapeamento de pontos base (String -> Ponto numérico)
                dados_mapa = self.df.iloc[5:].copy()
                dados_mapa[pts_col_name] = pd.to_numeric(dados_mapa[pts_col_name], errors='coerce')
                dados_mapa = dados_mapa.dropna(subset=[col_string, pts_col_name])
                valores_string_base = pd.Series(dados_mapa[pts_col_name].values, index=dados_mapa[col_string]).to_dict()

                if not valores_string_base:
                    print(f"Aviso: Mapeamento de pontos base para '{col_string}' vazio. Pulando gráfico.")
                    continue

            except Exception as e:
                print(f"Erro ao criar mapeamento para gráfico de string '{col_string}': {e}.")
                continue

            df_string = self.calculo_df[[modelo_col, col_string]].copy()
            df_string = df_string[~df_string[modelo_col].isin(['BOM', 'NEUTRO'])]

            if df_string.empty:
                print(f"Nenhum dado para plotar no gráfico de string para '{col_string}'.")
                continue

            # Mapeia para obter o PONTO BASE para o gráfico (não a pontuação final ponderada)
            df_string['ValorBase'] = df_string[col_string].map(valores_string_base).fillna(0)
            df_string = df_string.sort_values('ValorBase', ascending=False)
            df_string['Posicao'] = range(len(df_string)) # Posição X baseada na ordenação

            fig = px.scatter(df_string,
                            x='Posicao',
                            y='ValorBase', # Usa o valor base no eixo Y
                            color=col_string, # Cor baseada na categoria string
                            hover_name=modelo_col, # O que aparece ao passar o mouse
                            title=f'Comparação de {col_string.capitalize()} entre Modelos (Pontuação Base)',
                            size_max=15, # Tamanho dos pontos
                            hover_data={'Posicao': False, 'ValorBase': True, col_string: True},
                            labels={'ValorBase': 'Pontuação Base (do CSV)', col_string: col_string.capitalize()})

            # Adiciona linhas BOM/NEUTRO com base nos valores BASE do mapeamento
            valor_bom_str = self.valores_bom.get(col_string)
            valor_neutro_str = self.valores_neutro.get(col_string)
            valor_base_bom = valores_string_base.get(valor_bom_str, None) # Busca ponto base do BOM
            valor_base_neutro = valores_string_base.get(valor_neutro_str, None) # Busca ponto base do NEUTRO

            if valor_base_bom is not None:
                fig.add_hline(y=valor_base_bom, line_dash='dash',
                          line_color='green', line_width=2,
                          annotation_text=f"BOM: '{valor_bom_str}' ({valor_base_bom:.2f})",
                          annotation_position="top right",
                          annotation_font=dict(color='green', size=12))

            if valor_base_neutro is not None:
                fig.add_hline(y=valor_base_neutro, line_dash='dash',
                          line_color='orange', line_width=2,
                          annotation_text=f"NEUTRO: '{valor_neutro_str}' ({valor_base_neutro:.2f})",
                          annotation_position="bottom right",
                          annotation_font=dict(color='orange', size=12))

            fig.update_layout(
                xaxis=dict(title='Modelos Ordenados por Pontuação Base', showticklabels=False, showgrid=False, zeroline=False),
                yaxis=dict(title='Pontuação Base (Definida no CSV)', showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='lightgray'),
                plot_bgcolor='white', paper_bgcolor='white',
                margin=dict(l=40, r=40, t=60, b=40),
                legend=dict(title_text=col_string.capitalize(), orientation='h', yanchor='bottom', y=-0.3, xanchor='center', x=0.5),
                hoverlabel=dict(bgcolor='white', font_size=12, font_family='Arial')
            )

            # CORREÇÃO: Removido o parâmetro 'selector' que causava o erro
            fig.update_traces(marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey'), opacity=0.8))

            fig.show()

    def _gerar_grafico_booleanos(self):
        """Gráfico para todas as variáveis booleanas"""
        # Identifica colunas booleanas
        bool_cols = [col for col in self.tipos_dados.index
                    if self.tipos_dados[col] == 'boolean' and col in self.calculo_df.columns]

        if not bool_cols:
            return

        modelo_col = self.calculo_df.columns[0]

        for col_bool in bool_cols:
            df_bool = self.calculo_df[~self.calculo_df[modelo_col].isin(['BOM', 'NEUTRO'])][[modelo_col, col_bool]].copy()
            df_bool['Valor'] = df_bool[col_bool].map({True: 1, False: 0})
            df_bool['Tamanho'] = 20

            fig = px.scatter(df_bool,
                    x=modelo_col,
                    y='Valor',
                    color=col_bool,
                    color_discrete_map={True: '#4CAF50', False: '#F44336'},
                    size='Tamanho',
                    title=f'{col_bool} por Modelo',
                    hover_name=modelo_col,
                    hover_data={'Valor': False, 'Tamanho': False},
                    category_orders={col_bool: [True, False]})

            # Atualiza a legenda para mostrar SIM/NÃO
            fig.for_each_trace(lambda t: t.update(name='SIM' if t.name == 'True' else 'NÃO'))

            fig.update_yaxes(
                range=[-0.5, 1.5],
                tickvals=[0, 1],
                ticktext=['Não', 'Sim'],
                showgrid=False
            )

            # Adiciona linhas de referência SIM/NÃO
            fig.add_hline(y=1, line_dash='dash', line_color='green')
            fig.add_hline(y=0, line_dash='dash', line_color='orange')

            # Altera aparência do gráfico
            fig.update_layout(
                xaxis_title=None,
                yaxis_title=None,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )

            # Linhas de referência
            valor_bom = 1 if self.valores_bom[col_bool] else 0
            valor_neutro = 1 if self.valores_neutro[col_bool] else 0

            fig.show()

    def recomendar_produtos(self):
        """Recomenda produtos com base na pontuação incluindo todos os critérios"""
        # Assume que a primeira coluna é o identificador 'Modelo'
        modelo_col = self.df.columns[0]
        if 'Pontuacao_Total' not in self.calculo_df.columns:
            print("Pontuação total não calculada. Não é possível recomendar.")
            return

        produtos = self.calculo_df[~self.calculo_df[modelo_col].isin(['BOM', 'NEUTRO'])]

        if produtos.empty:
            print("Nenhum produto encontrado para recomendação (após excluir BOM/NEUTRO).")
            return

        # Verifica se há pontuações válidas
        if produtos['Pontuacao_Total'].isna().all():
            print("Todas as pontuações totais são inválidas. Não é possível recomendar.")
            return

        pontuacao_maxima = produtos['Pontuacao_Total'].max()
        # Lida com o caso de todas as pontuações serem NaN ou não haver máximo
        if pd.isna(pontuacao_maxima):
             print("Não foi possível determinar a pontuação máxima válida.")
             return

        recomendados = produtos[produtos['Pontuacao_Total'] == pontuacao_maxima]

        print("\n--- Produtos Recomendados ---")
        if recomendados.empty:
             print("Nenhum produto atingiu a pontuação máxima (pode indicar erro ou dados uniformes).")
             return

        for _, produto in recomendados.iterrows():
            print(f"\nProduto: {produto[modelo_col]} com pontuação total de {produto['Pontuacao_Total']:.2f}")
            print("Detalhes da Pontuação por Critério (Ordenado por Peso):")

            # Ordena critérios por peso (do maior para o menor)
            # Garante que o peso existe antes de tentar ordenar
            criterios_ordenados = sorted(
                [(col, self.pesos[col]) for col in self.pesos.index if col in self.pesos],
                key=lambda item: item[1], reverse=True
            )

            for col, peso in criterios_ordenados:
                pont_col = f"{col}_pontuacao"
                # Verifica se a coluna de pontuação existe no produto e tem valor válido
                if pont_col not in produto or pd.isna(produto[pont_col]):
                    # print(f"  - {col}: Pontuação não disponível ou inválida.")
                    continue # Pula se não houver pontuação para este critério

                pontuacao = produto[pont_col]
                valor_atual = produto.get(col, "N/A") # Pega o valor original com segurança
                tipo = self.tipos_dados.get(col, "desconhecido")

                # Formata a saída para melhor leitura
                justificativa = f"(Peso: {peso:.2f}, Pontos: {pontuacao:.2f})"

                if tipo == 'string':
                    valor_atual_str = f"'{valor_atual}'" if valor_atual != "N/A" else valor_atual
                    if pontuacao > 0:
                        print(f"  - {col}: {valor_atual_str} [Vantagem] {justificativa}")
                    elif pontuacao < 0:
                         print(f"  - {col}: {valor_atual_str} [Desvantagem] {justificativa}")
                    else:
                         print(f"  - {col}: {valor_atual_str} [Neutro] {justificativa}")

                elif tipo == 'boolean':
                     valor_atual_str = "Sim" if valor_atual == True else ("Não" if valor_atual == False else "N/A")
                     if pontuacao > 0:
                          print(f"  - {col}: {valor_atual_str} [Vantagem] {justificativa}")
                     elif pontuacao < 0:
                          print(f"  - {col}: {valor_atual_str} [Desvantagem] {justificativa}")
                     else:
                          print(f"  - {col}: {valor_atual_str} [Neutro] {justificativa}")

                elif tipo == 'number':
                     valor_atual_str = f"{valor_atual:.1f}" if isinstance(valor_atual, (int, float)) else str(valor_atual)
                     valor_bom = self.valores_bom.get(col)
                     valor_neutro = self.valores_neutro.get(col)

                     desc = f"{valor_atual_str}"
                     if pd.notna(valor_bom) and pd.notna(valor_neutro):
                          desc += f" (Bom: {valor_bom:.1f}, Neutro: {valor_neutro:.1f})"

                     if pontuacao >= peso * 0.99: # Praticamente atingiu o máximo
                          print(f"  - {col}: {desc} [Ótimo] {justificativa}")
                     elif pontuacao > 0:
                          print(f"  - {col}: {desc} [Bom] {justificativa}")
                     elif pontuacao == 0:
                           print(f"  - {col}: {desc} [Neutro] {justificativa}")
                     else: # pontuacao < 0
                          print(f"  - {col}: {desc} [Abaixo do Neutro] {justificativa}")
                else:
                     # Caso tipo desconhecido
                     print(f"  - {col}: {valor_atual} [Tipo Desconhecido] {justificativa}")


# --- Exemplo de Uso - Notebook ---
# Certifique-se que o arquivo 'base_eletronicos_2.csv' está no mesmo diretório
# ou forneça o caminho completo.
try:
    print("--- Análise para base de eletrônicos ---")
    analyzer_eletro = DataAnalyzer('base_eletronicos_2.csv')
    analyzer_eletro.load_and_prepare_data()
    analyzer_eletro.calcular_pontuacoes()
    analyzer_eletro.gerar_graficos()
    analyzer_eletro.recomendar_produtos()
except FileNotFoundError:
    print("\nErro: Arquivo 'base_eletronicos_2.csv' não encontrado.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado durante a análise: {e}")
    import traceback
    traceback.print_exc()

# --- Exemplo de Uso - Carro ---
# Certifique-se que o arquivo 'base_eletronicos_2.csv' está no mesmo diretório
# ou forneça o caminho completo.

# try:
#     print("--- Análise para base de carros ---")
#     analyzer_carro = DataAnalyzer('base_carro.csv')
#     analyzer_carro.load_and_prepare_data()
#     analyzer_carro.calcular_pontuacoes()
#     analyzer_carro.gerar_graficos()
#     analyzer_carro.recomendar_produtos()
# except FileNotFoundError:
#     print("\nErro: Arquivo 'base_carro.csv' não encontrado.")
# except Exception as e:
#     print(f"\nOcorreu um erro inesperado durante a análise: {e}")
#     import traceback
#     traceback.print_exc()

--- Análise para base de eletrônicos ---
Coluna string 'Cor' mapeada para 'Cor,pts_string' (original: 'Unnamed: 6')
Aviso: Coluna 'Unnamed: 6' definida nos tipos mas não encontrada nos dados para cálculo.
Colunas disponíveis para cálculo: ['Modelo', 'Memória RAM', 'Armazenamento', 'Tamanho de tela', 'Bateria mAh', 'Cor', 'Cor,pts_string', 'Bluetooth']
Pesos aplicáveis: {'Memória RAM': 0.3, 'Armazenamento': 0.2, 'Tamanho de tela': 0.1, 'Bateria mAh': 0.2, 'Cor': 0.1, 'Bluetooth': 0.1}
Tipos de Dados: {'Memória RAM': 'number', 'Armazenamento': 'number', 'Tamanho de tela': 'number', 'Bateria mAh': 'number', 'Cor': 'string', 'Unnamed: 6': 'pts_string', 'Bluetooth': 'boolean'}

Processando coluna: Memória RAM
  Tipo: number, Proporcionalidade: proportional
  Valores BOM: 8, NEUTRO: 4, Peso: 0.3
  Resultado parcial (Top 5):
 Memória RAM  Memória RAM_pontuacao
           2                  -0.15
           4                   0.00
           6                   0.15
           8              

Gerando gráfico para coluna numérica: 'Memória RAM'
  Gráfico de Memória RAM pronto para exibição.


Gerando gráfico para coluna numérica: 'Armazenamento'
  Gráfico de Armazenamento pronto para exibição.


Gerando gráfico para coluna numérica: 'Tamanho de tela'
  Gráfico de Tamanho de tela pronto para exibição.


Gerando gráfico para coluna numérica: 'Bateria mAh'
  Gráfico de Bateria mAh pronto para exibição.


Gerando gráfico para string: 'Cor'


--- Gráficos Gerados ---

--- Produtos Recomendados ---

Produto: Apple com pontuação total de 0.81
Detalhes da Pontuação por Critério (Ordenado por Peso):
  - Memória RAM: 16.0 (Bom: 8.0, Neutro: 4.0) [Ótimo] (Peso: 0.30, Pontos: 0.30)
  - Armazenamento: 1000.0 (Bom: 960.0, Neutro: 120.0) [Ótimo] (Peso: 0.20, Pontos: 0.20)
  - Bateria mAh: 6000.0 (Bom: 5000.0, Neutro: 3000.0) [Ótimo] (Peso: 0.20, Pontos: 0.20)
  - Tamanho de tela: 16.0 (Bom: 14.0, Neutro: 16.0) [Neutro] (Peso: 0.10, Pontos: 0.00)
  - Cor: 'Cinza' [Vantagem] (Peso: 0.10, Pontos: 0.01)
  - Bluetooth: Sim [Vantagem] (Peso: 0.10, Pontos: 0.10)


# Testing

## DeepSeek Refactor


In [9]:
"""
Módulo de análise de dados para comparação de produtos.

Este módulo permite carregar, analisar e visualizar dados de comparação de produtos
com base em critérios pré-definidos, gerando pontuações e recomendações.
"""

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from typing import Dict, Optional, List, Union, Any


class DataAnalyzer:
    """Classe principal para análise e comparação de dados de produtos."""

    def __init__(self, file_path: str) -> None:
        """Inicializa o analisador com o caminho do arquivo de dados.

        Args:
            file_path: Caminho para o arquivo CSV contendo os dados
        """
        self.file_path = file_path
        self.df = None
        self.weights = None
        self.data_types = None
        self.proportionality = None
        self.good_values = None
        self.neutral_values = None
        self.calculation_df = None
        self.string_columns_map = {}
        self.model_column = None

    def load_and_prepare_data(self) -> None:
        """Carrega e prepara os dados para análise."""
        self._load_raw_data()
        self._clean_empty_columns()
        self._extract_configurations()
        self._map_string_columns()
        self._extract_reference_values()
        self._prepare_calculation_data()
        self._convert_data_types()

    def _load_raw_data(self) -> None:
        """Carrega os dados brutos do arquivo CSV."""
        try:
            self.df = pd.read_csv(
                self.file_path,
                encoding='UTF-8',
                sep=',',
                skipinitialspace=True
            )
            self.model_column = self.df.columns[0]
        except Exception as e:
            raise ValueError(f"Erro ao carregar arquivo: {str(e)}")

    def _clean_empty_columns(self) -> None:
        """Remove colunas totalmente vazias do DataFrame."""
        self.df = self.df.dropna(axis=1, how='all')

    def _extract_configurations(self) -> None:
        """Extrai configurações de pesos, tipos de dados e proporcionalidade."""
        if len(self.df) < 3:
            raise ValueError("Arquivo CSV não contém linhas suficientes para configurações")

        self.weights = self.df.iloc[0].dropna().astype(float)
        self.data_types = self.df.iloc[1].dropna()
        self.proportionality = self.df.iloc[2].dropna()

    def _map_string_columns(self) -> None:
        """Mapeia colunas de string para suas colunas de pontos correspondentes."""
        self.string_columns_map = {}

        if self.data_types is None:
            raise ValueError("Tipos de dados não foram carregados corretamente")

        string_cols = [
            col for col in self.data_types.index
            if self.data_types[col] == 'string'
        ]

        for col in string_cols:
            try:
                col_idx = list(self.df.columns).index(col)
                if col_idx + 1 < len(self.df.columns):
                    pts_col = self.df.columns[col_idx + 1]
                    new_pts_col_name = f"{col}_points"
                    self.string_columns_map[col] = new_pts_col_name
                    self.df = self.df.rename(columns={pts_col: new_pts_col_name})
            except (ValueError, IndexError):
                print(f"Aviso: Não foi possível mapear coluna string '{col}'")

    def _extract_reference_values(self) -> None:
        """Extrai valores de referência BOM e NEUTRO."""
        good_row = self.df[self.df[self.model_column] == 'BOM']
        neutral_row = self.df[self.df[self.model_column] == 'NEUTRO']

        self.good_values = good_row.iloc[0].dropna() if not good_row.empty else pd.Series(dtype='object')
        self.neutral_values = neutral_row.iloc[0].dropna() if not neutral_row.empty else pd.Series(dtype='object')

        if good_row.empty or neutral_row.empty:
            print("Aviso: Linhas de referência 'BOM' ou 'NEUTRO' não encontradas")

    def _prepare_calculation_data(self) -> None:
        """Prepara o DataFrame para cálculos, removendo linhas de configuração."""
        self.calculation_df = self.df.iloc[5:].reset_index(drop=True)

    def _convert_data_types(self) -> None:
        """Converte os tipos de dados conforme especificado."""
        for col in self.data_types.index:
            if col not in self.calculation_df.columns:
                continue

            dtype = self.data_types[col]

            try:
                if dtype == 'number':
                    self._convert_numeric_column(col)
                elif dtype == 'boolean':
                    self._convert_boolean_column(col)
                elif dtype == 'string':
                    self.calculation_df[col] = self.calculation_df[col].astype(str)
                elif dtype == 'pts_string':
                    continue
                else:
                    print(f"Aviso: Tipo de dado desconhecido '{dtype}' para coluna '{col}'")
            except Exception as e:
                print(f"Erro ao converter coluna '{col}': {str(e)}")

    def _convert_numeric_column(self, col: str) -> None:
        """Converte coluna numérica e valores de referência."""
        self.good_values[col] = pd.to_numeric(self.good_values.get(col), errors='coerce')
        self.neutral_values[col] = pd.to_numeric(self.neutral_values.get(col), errors='coerce')
        self.calculation_df[col] = pd.to_numeric(self.calculation_df[col], errors='coerce')

    def _convert_boolean_column(self, col: str) -> None:
        """Converte coluna booleana e valores de referência."""
        self.good_values[col] = str(self.good_values.get(col, '')).strip().upper() == 'TRUE'
        self.neutral_values[col] = str(self.neutral_values.get(col, '')).strip().upper() == 'TRUE'
        self.calculation_df[col] = (
            self.calculation_df[col]
            .astype(str)
            .str.strip()
            .str.upper()
            .map({'TRUE': True, 'FALSE': False})
            .fillna(False)
            .astype(bool)
        )

    def calculate_scores(self) -> None:
        """Calcula pontuações para todos os critérios."""
        self._validate_calculation_preconditions()

        print("Colunas disponíveis para cálculo:", list(self.calculation_df.columns))
        print("Pesos aplicáveis:", self.weights.to_dict())
        print("Tipos de Dados:", self.data_types.to_dict())

        for col in self.weights.index:
            self._process_column_for_scoring(col)

        self._calculate_total_score()

    def _validate_calculation_preconditions(self) -> None:
        """Verifica se os dados estão prontos para cálculo."""
        if self.calculation_df is None or self.data_types is None or self.weights is None:
            raise ValueError("Dados não carregados corretamente. Execute load_and_prepare_data() primeiro")

    def _process_column_for_scoring(self, col: str) -> None:
        """Processa uma coluna individual para cálculo de pontuação."""
        print(f"\nProcessando coluna: {col}")

        score_col_name = f"{col}_score"
        dtype = self.data_types.get(col)

        try:
            if dtype in ['number', 'boolean']:
                self._calculate_numeric_score(col, score_col_name)
            elif dtype == 'string':
                self._calculate_string_score(col, score_col_name)

            self._show_partial_results(col, score_col_name)
        except Exception as e:
            print(f"Erro ao processar coluna '{col}': {str(e)}")
            self.calculation_df[score_col_name] = 0

    def _calculate_numeric_score(self, col: str, score_col_name: str) -> None:
        """Calcula pontuação para coluna numérica ou booleana."""
        good_value = self.good_values.get(col)
        neutral_value = self.neutral_values.get(col)
        weight = self.weights.get(col, 0)
        proportionality = self.proportionality.get(col)

        if pd.isna(good_value) or pd.isna(neutral_value) or not proportionality:
            print(f"Aviso: Valores de referência ausentes para '{col}'. Pontuação zerada.")
            self.calculation_df[score_col_name] = 0
            return

        print(f"  Tipo: {self.data_types[col]}, Proporcionalidade: {proportionality}")
        print(f"  Valores BOM: {good_value}, NEUTRO: {neutral_value}, Peso: {weight}")

        self.calculation_df[score_col_name] = self.calculation_df.apply(
            lambda row: self._calculate_numeric_score_value(
                row[col], good_value, neutral_value, weight, proportionality
            ) if col in row else 0,
            axis=1
        )

    def _calculate_string_score(self, col: str, score_col_name: str) -> None:
        """Calcula pontuação para coluna de string."""
        if col not in self.calculation_df.columns:
            print(f"Erro: Coluna string '{col}' não encontrada")
            self.calculation_df[score_col_name] = 0
            return

        pts_col = self.string_columns_map.get(col)
        print(f"  Tipo: string. Coluna de pontos esperada: {pts_col}. Peso: {self.weights.get(col, 0)}")

        self.calculation_df[score_col_name] = self.calculation_df.apply(
            lambda row: self._calculate_string_score_value(row[col], col),
            axis=1
        )

    def _show_partial_results(self, col: str, score_col_name: str) -> None:
        """Mostra resultados parciais para uma coluna."""
        if score_col_name in self.calculation_df:
            print(f"  Resultado parcial (Top 5):\n{self.calculation_df[[col, score_col_name]].head().to_string(index=False)}")
        else:
            print(f"  Aviso: Coluna de pontuação '{score_col_name}' não foi criada.")

    def _calculate_total_score(self) -> None:
        """Calcula a pontuação total somando todas as pontuações individuais."""
        score_columns = [c for c in self.calculation_df.columns if c.endswith('_score')]

        if not score_columns:
            print("\nNenhuma coluna de pontuação gerada. Pontuação total não calculada.")
            self.calculation_df['Total_Score'] = 0
        else:
            print(f"\nColunas usadas para Pontuação Total: {score_columns}")
            self.calculation_df['Total_Score'] = self.calculation_df[score_columns].sum(axis=1)

        print("\nResumo final das pontuações:")
        cols_to_show = [self.model_column] + score_columns + ['Total_Score']
        valid_cols = [c for c in cols_to_show if c in self.calculation_df.columns]
        print(self.calculation_df[valid_cols].head().to_string(index=False))

    def _calculate_numeric_score_value(
        self,
        value: Union[float, bool],
        good_value: Union[float, bool],
        neutral_value: Union[float, bool],
        weight: float,
        proportionality_type: str
    ) -> float:
        """Calcula a pontuação para um valor numérico ou booleano individual."""
        # Tratamento para valores booleanos
        if isinstance(value, (bool, np.bool_)):
            return self._calculate_boolean_score(value, good_value, neutral_value, weight)

        # Verificação de valores inválidos
        if pd.isna(value) or pd.isna(good_value) or pd.isna(neutral_value) or pd.isna(weight):
            return 0

        # Caso onde bom == neutro
        if good_value == neutral_value:
            return 0

        # Lógica de cálculo proporcional
        if proportionality_type == 'proportional':
            return self._calculate_proportional_score(value, good_value, neutral_value, weight)
        elif proportionality_type == 'i_proportional':
            return self._calculate_inverse_proportional_score(value, good_value, neutral_value, weight)

        return 0

    def _calculate_boolean_score(
        self,
        value: bool,
        good_value: bool,
        neutral_value: bool,
        weight: float
    ) -> float:
        """Calcula pontuação para valores booleanos."""
        if good_value == neutral_value:
            return weight if value == good_value else 0
        else:
            if value == good_value:
                return weight
            elif value == neutral_value:
                return 0
            else:
                return -weight

    def _calculate_proportional_score(
        self,
        value: float,
        good_value: float,
        neutral_value: float,
        weight: float
    ) -> float:
        """Calcula pontuação proporcional."""
        if good_value > neutral_value:  # Caso crescente
            if value >= good_value:
                return weight
            elif value >= neutral_value:
                return ((value - neutral_value) / (good_value - neutral_value)) * weight
            else:
                denominator = neutral_value if neutral_value != 0 else 1
                return -((abs(neutral_value - value)) / abs(denominator)) * weight
        elif good_value < neutral_value:  # Caso decrescente
            if value <= good_value:
                return weight
            elif value <= neutral_value:
                return ((value - good_value) / (neutral_value - good_value)) * weight
            else:
                denominator = neutral_value if neutral_value != 0 else 1
                return -((abs(value - neutral_value)) / abs(denominator)) * weight
        else:
            return 0

    def _calculate_inverse_proportional_score(
        self,
        value: float,
        good_value: float,
        neutral_value: float,
        weight: float
    ) -> float:
        """Calcula pontuação inversamente proporcional."""
        if good_value < neutral_value:  # Caso decrescente normal
            if value <= good_value:
                return weight
            elif value <= neutral_value:
                return ((neutral_value - value) / (neutral_value - good_value)) * weight
            else:
                denominator = neutral_value if neutral_value != 0 else 1
                return -((abs(value - neutral_value)) / abs(denominator)) * weight
        elif good_value > neutral_value:  # Caso crescente incomum
            if value >= good_value:
                return weight
            elif value >= neutral_value:
                return ((good_value - value) / (good_value - neutral_value)) * weight
            else:
                denominator = neutral_value if neutral_value != 0 else 1
                return -((abs(neutral_value - value)) / abs(denominator)) * weight
        else:
            return 0

    def _calculate_string_score_value(self, value: str, column: str) -> float:
        """Calcula pontuação para um valor de string individual."""
        points_column = self.string_columns_map.get(column)

        if not points_column or points_column not in self.df.columns:
            return 0

        try:
            # Cria mapeamento de valores string para pontos
            mapping_data = self.df.iloc[5:].copy()
            mapping_data[points_column] = pd.to_numeric(mapping_data[points_column], errors='coerce')
            mapping_data = mapping_data.dropna(subset=[column, points_column])

            mapping = pd.Series(
                mapping_data[points_column].values,
                index=mapping_data[column]
            ).to_dict()

            if not mapping:
                return 0

            # Calcula pontuação base * peso
            clean_value = str(value).strip()
            base_score = mapping.get(clean_value, 0)
            column_weight = float(self.weights.get(column, 0))

            return base_score * column_weight
        except Exception as e:
            print(f"Erro ao calcular pontuação para '{column}': {str(e)}")
            return 0

    def generate_visualizations(self) -> None:
        """Gera todas as visualizações de análise."""
        print("\n--- Gerando Visualizações ---")

        if self.calculation_df is None:
            print("Dados não disponíveis para geração de gráficos")
            return

        try:
            self._generate_total_score_chart()
            self._generate_numeric_charts()
            self._generate_string_charts()
            self._generate_boolean_charts()
            print("--- Visualizações Geradas ---")
        except Exception as e:
            print(f"Erro ao gerar visualizações: {str(e)}")

    def _generate_total_score_chart(self) -> None:
        """Gera gráfico de barras para pontuação total."""
        if 'Total_Score' not in self.calculation_df.columns:
            print("Pontuação total não calculada. Pulando gráfico.")
            return

        max_theoretical_score = self.weights.sum() if self.weights is not None else 1.0
        print(f"  Pontuação máxima teórica: {max_theoretical_score:.2f}")

        analysis_df = self.calculation_df[
            ~self.calculation_df[self.model_column].isin(['BOM', 'NEUTRO'])
        ].copy()

        analysis_df.dropna(subset=['Total_Score'], inplace=True)

        if analysis_df.empty:
            print("Nenhum dado válido para plotar")
            return

        # Ordena e classifica por pontuação
        analysis_df = analysis_df.sort_values('Total_Score', ascending=False)
        analysis_df['Color_Category'] = analysis_df['Total_Score'].apply(
            lambda x: 'Positiva' if x >= 0 else 'Negativa'
        )

        # Cria gráfico
        fig = px.bar(
            analysis_df,
            x='Total_Score',
            y=self.model_column,
            orientation='h',
            color='Color_Category',
            color_discrete_map={'Positiva': 'lightgreen', 'Negativa': 'palevioletred'},
            hover_name=self.model_column,
            hover_data={'Total_Score': ':.2f'},
            title='Pontuação Total dos Modelos'
        )

        # Configurações de layout
        fig.update_layout(
            yaxis={'categoryorder': 'total ascending'},
            plot_bgcolor='white',
            paper_bgcolor='white',
            legend_title_text='Resultado',
            xaxis_title='Pontuação Total',
            yaxis_title='Modelo'
        )

        # Adiciona linhas de referência
        fig.add_vline(x=0, line_dash='dash', line_color='black')
        fig.add_vline(x=max_theoretical_score, line_dash='dash', line_color='black')

        # Adiciona anotações
        self._add_reference_annotation(
            fig, max_theoretical_score, len(analysis_df),
            "Desejável (BOM)<br>(NEUTRO)"
        )
        self._add_reference_annotation(
            fig, 0, len(analysis_df),
            "Mínimo Aceitável<br>(NEUTRO)"
        )

        # Adiciona anotação para melhor produto
        if not analysis_df.empty:
            best_product = analysis_df.iloc[0]
            fig.add_annotation(
                x=best_product['Total_Score'],
                y=best_product[self.model_column],
                text="🏆 Melhor Produto",
                showarrow=False,
                font=dict(color="#00008B", size=12),
                bgcolor="white",
                bordercolor="black",
                borderwidth=1,
                borderpad=4,
                xshift=-15,
                yshift=25
            )

        fig.show()

    def _add_reference_annotation(
        self,
        fig: go.Figure,
        x_value: float,
        y_position: int,
        text: str
    ) -> None:
        """Adiciona anotação de referência ao gráfico."""
        fig.add_annotation(
            x=x_value,
            y=y_position - 1,
            xref="x",
            yref="y",
            text=text,
            showarrow=True,
            font=dict(size=16, color="#000000"),
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

    def _generate_numeric_charts(self) -> None:
        """Gera gráficos para colunas numéricas."""
        if self.data_types is None:
            return

        numeric_cols = [
            col for col in self.data_types.index
            if self.data_types[col] == 'number' and col in self.calculation_df.columns
        ]

        for col in numeric_cols:
            print(f"Gerando gráfico para coluna numérica: '{col}'")

            # Prepara dados
            df = self.calculation_df[
                ~self.calculation_df[self.model_column].isin(['BOM', 'NEUTRO'])
            ][[self.model_column, col]].copy()

            df.dropna(subset=[col], inplace=True)

            if df.empty:
                continue

            # Obtém valores de referência
            good_value = self.good_values.get(col)
            neutral_value = self.neutral_values.get(col)
            is_inverse = self.proportionality.get(col, '').lower() == 'i_proportional'

            # Classifica valores
            df['Color_Category'] = df.apply(
                lambda x: self._classify_numeric_value(
                    x[col], good_value, neutral_value, is_inverse
                ),
                axis=1
            )

            # Ordena
            df = df.sort_values(col, ascending=not is_inverse)

            # Cria gráfico
            fig = px.bar(
                df,
                x=col,
                y=self.model_column,
                orientation='h',
                color='Color_Category',
                color_discrete_map={
                    'Positiva': 'lightgreen',
                    'Negativa': 'palevioletred',
                    'Neutra': 'lightgray'
                },
                hover_name=self.model_column,
                hover_data={col: ':.2f'},
                title=f"{col} {'(Inversamente Proporcional)' if is_inverse else ''}"
            )

            # Configura layout
            fig.update_layout(
                plot_bgcolor='white',
                paper_bgcolor='white',
                legend_title_text='Resultado',
                xaxis_title=col,
                yaxis_title='Modelo'
            )

            # Adiciona linhas de referência
            if pd.notna(neutral_value):
                self._add_reference_line(
                    fig, float(neutral_value), len(df),
                    "Mínimo Aceitável<br>(NEUTRO)",
                    is_inverse
                )
            if pd.notna(good_value):
                self._add_reference_line(
                    fig, float(good_value), len(df),
                    "Desejável (BOM)",
                    is_inverse
                )

            fig.show()

    def _classify_numeric_value(
        self,
        value: float,
        good_value: float,
        neutral_value: float,
        is_inverse: bool
    ) -> str:
        """Classifica um valor numérico como Positivo, Negativo ou Neutro."""
        if pd.isna(neutral_value):
            return 'Neutra'

        value = float(value)
        neutral_value = float(neutral_value)

        if is_inverse:
            return 'Positiva' if value <= neutral_value else 'Negativa'
        else:
            return 'Positiva' if value >= neutral_value else 'Negativa'

    def _add_reference_line(
        self,
        fig: go.Figure,
        value: float,
        y_position: int,
        text: str,
        is_inverse: bool
    ) -> None:
        """Adiciona linha de referência ao gráfico numérico."""
        fig.add_vline(
            x=value,
            line_dash='dash',
            line_color='black'
        )

        fig.add_annotation(
            x=value,
            y=y_position - 1,
            text=text,
            showarrow=True,
            font=dict(size=16, color="#000000"),
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

    def _generate_string_charts(self) -> None:
        """Gera gráficos para colunas de string."""
        if self.data_types is None:
            return

        string_cols = [
            col for col in self.data_types.index
            if self.data_types.get(col) == 'string' and col in self.calculation_df.columns
        ]

        for col in string_cols:
            print(f"Gerando gráfico para string: '{col}'")
            pts_col = self.string_columns_map.get(col)

            if not pts_col or pts_col not in self.df.columns:
                continue

            try:
                # Cria mapeamento string -> pontos
                mapping_data = self.df.iloc[5:].copy()
                mapping_data[pts_col] = pd.to_numeric(mapping_data[pts_col], errors='coerce')
                mapping_data = mapping_data.dropna(subset=[col, pts_col])

                string_mapping = pd.Series(
                    mapping_data[pts_col].values,
                    index=mapping_data[col]
                ).to_dict()

                if not string_mapping:
                    continue

                # Prepara dados para plotagem
                df = self.calculation_df[
                    ~self.calculation_df[self.model_column].isin(['BOM', 'NEUTRO'])
                ][[self.model_column, col]].copy()

                df['BaseValue'] = df[col].map(string_mapping).fillna(0)
                df = df.sort_values('BaseValue', ascending=False)
                df['Position'] = range(len(df))

                # Cria gráfico
                fig = px.scatter(
                    df,
                    x='Position',
                    y='BaseValue',
                    color=col,
                    hover_name=self.model_column,
                    title=f'Comparação de {col.capitalize()} entre Modelos (Pontuação Base)',
                    size_max=15,
                    hover_data={'Position': False, 'BaseValue': True, col: True},
                    labels={
                        'BaseValue': 'Pontuação Base (do CSV)',
                        col: col.capitalize()
                    }
                )

                # Adiciona linhas de referência
                good_str = self.good_values.get(col)
                neutral_str = self.neutral_values.get(col)
                good_base = string_mapping.get(good_str)
                neutral_base = string_mapping.get(neutral_str)

                if good_base is not None:
                    fig.add_hline(
                        y=good_base,
                        line_dash='dash',
                        line_color='green',
                        line_width=2,
                        annotation_text=f"BOM: '{good_str}' ({good_base:.2f})",
                        annotation_position="top right",
                        annotation_font=dict(color='green', size=12)
                    )

                if neutral_base is not None:
                    fig.add_hline(
                        y=neutral_base,
                        line_dash='dash',
                        line_color='orange',
                        line_width=2,
                        annotation_text=f"NEUTRO: '{neutral_str}' ({neutral_base:.2f})",
                        annotation_position="bottom right",
                        annotation_font=dict(color='orange', size=12)
                    )

                # Configura layout
                fig.update_layout(
                    xaxis=dict(
                        title='Modelos Ordenados por Pontuação Base',
                        showticklabels=False,
                        showgrid=False,
                        zeroline=False
                    ),
                    yaxis=dict(
                        title='Pontuação Base (Definida no CSV)',
                        showgrid=True,
                        gridcolor='lightgray',
                        zeroline=True,
                        zerolinecolor='lightgray'
                    ),
                    plot_bgcolor='white',
                    paper_bgcolor='white',
                    margin=dict(l=40, r=40, t=60, b=40),
                    legend=dict(
                        title_text=col.capitalize(),
                        orientation='h',
                        yanchor='bottom',
                        y=-0.3,
                        xanchor='center',
                        x=0.5
                    ),
                    hoverlabel=dict(
                        bgcolor='white',
                        font_size=12,
                        font_family='Arial'
                    )
                )

                fig.update_traces(
                    marker=dict(
                        size=12,
                        line=dict(width=1, color='DarkSlateGrey'),
                        opacity=0.8
                    )
                )

                fig.show()
            except Exception as e:
                print(f"Erro ao gerar gráfico para '{col}': {str(e)}")

    def _generate_boolean_charts(self) -> None:
        """Gera gráficos para colunas booleanas."""
        if self.data_types is None:
            return

        bool_cols = [
            col for col in self.data_types.index
            if self.data_types[col] == 'boolean' and col in self.calculation_df.columns
        ]

        for col in bool_cols:
            df = self.calculation_df[
                ~self.calculation_df[self.model_column].isin(['BOM', 'NEUTRO'])
            ][[self.model_column, col]].copy()

            df['Value'] = df[col].map({True: 1, False: 0})
            df['Size'] = 20

            fig = px.scatter(
                df,
                x=self.model_column,
                y='Value',
                color=col,
                color_discrete_map={True: '#4CAF50', False: '#F44336'},
                size='Size',
                title=f'{col} por Modelo',
                hover_name=self.model_column,
                hover_data={'Value': False, 'Size': False},
                category_orders={col: [True, False]}
            )

            # Atualiza a legenda
            fig.for_each_trace(
                lambda t: t.update(name='SIM' if t.name == 'True' else 'NÃO')
            )

            # Configura eixos
            fig.update_yaxes(
                range=[-0.5, 1.5],
                tickvals=[0, 1],
                ticktext=['Não', 'Sim'],
                showgrid=False
            )

            # Adiciona linhas de referência
            fig.add_hline(y=1, line_dash='dash', line_color='green')
            fig.add_hline(y=0, line_dash='dash', line_color='orange')

            # Configura layout
            fig.update_layout(
                xaxis_title=None,
                yaxis_title=None,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )

            fig.show()

    def recommend_products(self) -> None:
        """Recomenda produtos com base na análise realizada."""
        if 'Total_Score' not in self.calculation_df.columns:
            print("Pontuação total não calculada. Não é possível recomendar.")
            return

        products = self.calculation_df[
            ~self.calculation_df[self.model_column].isin(['BOM', 'NEUTRO'])
        ]

        if products.empty:
            print("Nenhum produto encontrado para recomendação.")
            return

        max_score = products['Total_Score'].max()

        if pd.isna(max_score):
            print("Não foi possível determinar a pontuação máxima.")
            return

        recommended = products[products['Total_Score'] == max_score]

        print("\n--- Produtos Recomendados ---")

        if recommended.empty:
            print("Nenhum produto atingiu a pontuação máxima.")
            return

        for _, product in recommended.iterrows():
            self._print_product_recommendation(product)

    def _print_product_recommendation(self, product: pd.Series) -> None:
        """Imprime detalhes da recomendação de um produto."""
        print(f"\nProduto: {product[self.model_column]} com pontuação total de {product['Total_Score']:.2f}")
        print("Detalhes da Pontuação por Critério (Ordenado por Peso):")

        # Ordena critérios por peso
        sorted_criteria = sorted(
            [(col, self.weights[col]) for col in self.weights.index if col in self.weights],
            key=lambda x: x[1], reverse=True
        )

        for col, weight in sorted_criteria:
            score_col = f"{col}_score"

            if score_col not in product or pd.isna(product[score_col]):
                continue

            score = product[score_col]
            current_value = product.get(col, "N/A")
            dtype = self.data_types.get(col, "desconhecido")

            self._print_criterion_details(col, weight, score, current_value, dtype)

    def _print_criterion_details(
        self,
        criterion: str,
        weight: float,
        score: float,
        current_value: Union[str, float, bool],
        dtype: str
    ) -> None:
        """Imprime detalhes de um critério individual."""
        justification = f"(Peso: {weight:.2f}, Pontos: {score:.2f})"

        if dtype == 'string':
            value_str = f"'{current_value}'" if current_value != "N/A" else current_value
            if score > 0:
                print(f"  - {criterion}: {value_str} [Vantagem] {justification}")
            elif score < 0:
                print(f"  - {criterion}: {value_str} [Desvantagem] {justification}")
            else:
                print(f"  - {criterion}: {value_str} [Neutro] {justification}")

        elif dtype == 'boolean':
            value_str = self._format_boolean_value(current_value)
            if score > 0:
                print(f"  - {criterion}: {value_str} [Vantagem] {justification}")
            elif score < 0:
                print(f"  - {criterion}: {value_str} [Desvantagem] {justification}")
            else:
                print(f"  - {criterion}: {value_str} [Neutro] {justification}")

        elif dtype == 'number':
            value_str = self._format_numeric_value(current_value, criterion)
            if score >= weight * 0.99:
                print(f"  - {criterion}: {value_str} [Ótimo] {justification}")
            elif score > 0:
                print(f"  - {criterion}: {value_str} [Bom] {justification}")
            elif score == 0:
                print(f"  - {criterion}: {value_str} [Neutro] {justification}")
            else:
                print(f"  - {criterion}: {value_str} [Abaixo do Neutro] {justification}")
        else:
            print(f"  - {criterion}: {current_value} [Tipo Desconhecido] {justification}")

    def _format_boolean_value(self, value: Union[bool, str]) -> str:
        """Formata valor booleano para exibição."""
        if value is True:
            return "Sim"
        elif value is False:
            return "Não"
        else:
            return "N/A"

    def _format_numeric_value(self, value: Union[float, str], column: str) -> str:
        """Formata valor numérico para exibição com referências."""
        if isinstance(value, (int, float)):
            value_str = f"{value:.1f}"
        else:
            value_str = str(value)

        good_value = self.good_values.get(column)
        neutral_value = self.neutral_values.get(column)

        if pd.notna(good_value) and pd.notna(neutral_value):
            value_str += f" (Bom: {float(good_value):.1f}, Neutro: {float(neutral_value):.1f})"

        return value_str


def analyze_electronics_data(file_path: str = 'base_eletronicos_2.csv') -> None:
    """Executa análise completa para dados de eletrônicos."""
    try:
        print("--- Análise para base de eletrônicos ---")
        analyzer = DataAnalyzer(file_path)
        analyzer.load_and_prepare_data()
        analyzer.calculate_scores()
        analyzer.generate_visualizations()
        analyzer.recommend_products()
    except FileNotFoundError:
        print("\nErro: Arquivo não encontrado.")
    except Exception as e:
        print(f"\nOcorreu um erro inesperado: {str(e)}")


if __name__ == "__main__":
    analyze_electronics_data()

--- Análise para base de eletrônicos ---
Colunas disponíveis para cálculo: ['Modelo', 'Memória RAM', 'Armazenamento', 'Tamanho de tela', 'Bateria mAh', 'Cor', 'Cor_points', 'Bluetooth']
Pesos aplicáveis: {'Memória RAM': 0.3, 'Armazenamento': 0.2, 'Tamanho de tela': 0.1, 'Bateria mAh': 0.2, 'Cor': 0.1, 'Bluetooth': 0.1}
Tipos de Dados: {'Memória RAM': 'number', 'Armazenamento': 'number', 'Tamanho de tela': 'number', 'Bateria mAh': 'number', 'Cor': 'string', 'Unnamed: 6': 'pts_string', 'Bluetooth': 'boolean'}

Processando coluna: Memória RAM
  Tipo: number, Proporcionalidade: proportional
  Valores BOM: 8, NEUTRO: 4, Peso: 0.3
  Resultado parcial (Top 5):
 Memória RAM  Memória RAM_score
           2              -0.15
           4               0.00
           6               0.15
           8               0.30
          16               0.30

Processando coluna: Armazenamento
  Tipo: number, Proporcionalidade: proportional
  Valores BOM: 960, NEUTRO: 120, Peso: 0.2
  Resultado parcial 

Gerando gráfico para coluna numérica: 'Memória RAM'


Gerando gráfico para coluna numérica: 'Armazenamento'


Gerando gráfico para coluna numérica: 'Tamanho de tela'


Gerando gráfico para coluna numérica: 'Bateria mAh'


Gerando gráfico para string: 'Cor'


--- Visualizações Geradas ---

--- Produtos Recomendados ---

Produto: Apple com pontuação total de 0.81
Detalhes da Pontuação por Critério (Ordenado por Peso):
  - Memória RAM: 16.0 (Bom: 8.0, Neutro: 4.0) [Ótimo] (Peso: 0.30, Pontos: 0.30)
  - Armazenamento: 1000.0 (Bom: 960.0, Neutro: 120.0) [Ótimo] (Peso: 0.20, Pontos: 0.20)
  - Bateria mAh: 6000.0 (Bom: 5000.0, Neutro: 3000.0) [Ótimo] (Peso: 0.20, Pontos: 0.20)
  - Tamanho de tela: 16.0 (Bom: 14.0, Neutro: 16.0) [Neutro] (Peso: 0.10, Pontos: 0.00)
  - Cor: 'Cinza' [Vantagem] (Peso: 0.10, Pontos: 0.01)
  - Bluetooth: Sim [Vantagem] (Peso: 0.10, Pontos: 0.10)


## Gemini Refactor

In [12]:
# -*- coding: utf-8 -*-
# [source: 151]
import pandas as pd
import plotly.express as px
import numpy as np
import logging
from typing import Optional, Dict, Any, List, Union, Tuple
from enum import Enum

# --- Constantes ---
CONFIG_ROW_PESOS = 0
CONFIG_ROW_TIPOS = 1
CONFIG_ROW_PROPORCIONALIDADE = 2
DATA_START_ROW = 5 # Linha onde começam os dados reais (após BOM/NEUTRO)

ROW_LABEL_BOM = 'BOM'
ROW_LABEL_NEUTRO = 'NEUTRO'

PONTUACAO_SUFFIX = '_pontuacao'
PTS_STRING_SUFFIX = ',pts_string' # Sufixo esperado na coluna de pontos de string no CSV

# Enum para tipos de dados (melhor que strings)
class DataType(Enum):
    NUMBER = 'number'
    BOOLEAN = 'boolean'
    STRING = 'string'
    PTS_STRING = 'pts_string' # Tipo especial para a coluna de pontos da string

# Enum para tipos de proporcionalidade
class Proportionality(Enum):
    PROPORTIONAL = 'proportional'
    I_PROPORTIONAL = 'i_proportional'

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

class DataAnalyzer:
    """
    Analisa dados de um arquivo CSV para realizar Análise Multicritério (MCDA),
    calculando pontuações, gerando gráficos e recomendando os melhores itens.

    A estrutura esperada do CSV é:
    Linha 0: Pesos (float) para cada critério.
    Linha 1: Tipos de dados ('number', 'boolean', 'string') para cada critério.
    Linha 2: Proporcionalidade ('proportional', 'i_proportional') para critérios numéricos.
    Linha 3: Valores considerados 'BOM' para cada critério.
    Linha 4: Valores considerados 'NEUTRO' para cada critério.
    Linha 5 adiante: Dados dos itens a serem analisados, com a primeira coluna sendo o identificador.
    Colunas 'string' devem ser seguidas imediatamente por uma coluna com seus pontos (nome original não importa, será renomeada).
    """

    def __init__(self, file_path: str):
# [source: 151]
        self.file_path: str = file_path
        self.df_original: Optional[pd.DataFrame] = None
        self.calculo_df: Optional[pd.DataFrame] = None
        self.pesos: Optional[pd.Series] = None
        self.tipos_dados: Optional[Dict[str, DataType]] = None
        self.proporcionalidade: Optional[Dict[str, Proportionality]] = None
        self.valores_bom: Optional[pd.Series] = None
        self.valores_neutro: Optional[pd.Series] = None
        # Mapeia nome da coluna string original para o nome *renomeado* da coluna de pontos
        self.string_columns_map: Dict[str, str] = {}
# [source: 152]
        # Mapeamento interno de strings para seus pontos base (criado sob demanda)
        self._string_to_points_base_map: Dict[str, Dict[str, float]] = {}

    def load_and_prepare_data(self) -> None:
        """
        Carrega os dados do CSV, extrai configurações, mapeia colunas de string,
        converte tipos de dados e prepara o DataFrame para cálculo.
        """
# [source: 152]
        logging.info(f"Carregando dados de: {self.file_path}")
        try:
            self.df_original = pd.read_csv(self.file_path, encoding='UTF-8', sep=',', skipinitialspace=True)
            self.df_original = self.df_original.dropna(axis=1, how='all') # Remove colunas totalmente vazias
        except FileNotFoundError:
            logging.error(f"Erro: Arquivo não encontrado em {self.file_path}")
            raise
        except Exception as e:
            logging.error(f"Erro ao ler o CSV: {e}")
            raise

        if self.df_original.empty:
            logging.warning("Arquivo CSV carregado está vazio.")
            return

        if len(self.df_original) < DATA_START_ROW + 1:
             logging.error(f"Arquivo CSV não contém linhas de configuração e/ou dados suficientes (necessita >= {DATA_START_ROW + 1} linhas).")
             self.df_original = None # Invalida o dataframe
             return

        self._extract_config()
        self._mapear_colunas_string() # Mapeia ANTES de remover as linhas de config/BOM/NEUTRO
        self._extract_bom_neutro()
        self._prepare_calculo_df()
        self._convert_data_types()

        logging.info("Dados carregados e preparados com sucesso.")


    def _extract_config(self) -> None:
        """Extrai pesos, tipos e proporcionalidade das linhas de configuração."""
        if self.df_original is None: return

        try:
# [source: 153]
            self.pesos = self.df_original.iloc[CONFIG_ROW_PESOS].dropna().astype(float)

            raw_tipos = self.df_original.iloc[CONFIG_ROW_TIPOS].dropna().astype(str).str.lower()
            self.tipos_dados = {col: DataType(tipo) for col, tipo in raw_tipos.items() if tipo in [dt.value for dt in DataType]}
            invalid_types = {col: tipo for col, tipo in raw_tipos.items() if tipo not in [dt.value for dt in DataType]}
            if invalid_types:
                logging.warning(f"Tipos de dados inválidos ignorados: {invalid_types}")


            raw_prop = self.df_original.iloc[CONFIG_ROW_PROPORCIONALIDADE].dropna().astype(str).str.lower()
            self.proporcionalidade = {col: Proportionality(prop) for col, prop in raw_prop.items() if prop in [p.value for p in Proportionality]}
            invalid_prop = {col: prop for col, prop in raw_prop.items() if prop not in [p.value for p in Proportionality]}
            if invalid_prop:
                logging.warning(f"Tipos de proporcionalidade inválidos ignorados: {invalid_prop}")

            logging.info(f"Pesos extraídos: {self.pesos.to_dict()}")
            logging.info(f"Tipos de dados extraídos: { {k: v.value for k, v in self.tipos_dados.items()} if self.tipos_dados else {} }")
            logging.info(f"Proporcionalidade extraída: { {k: v.value for k, v in self.proporcionalidade.items()} if self.proporcionalidade else {} }")

        except (IndexError, ValueError, TypeError) as e:
            logging.error(f"Erro ao extrair configuração das linhas iniciais: {e}")
            # Invalida as configurações se houver erro
            self.pesos = None
            self.tipos_dados = None
            self.proporcionalidade = None


    def _mapear_colunas_string(self) -> None:
        """
        Identifica colunas 'string' e mapeia-as para suas colunas de pontos
        adjacentes, renomeando a coluna de pontos para um padrão consistente.
        """
# [source: 179]
        if self.df_original is None or self.tipos_dados is None:
            logging.warning("DataFrame original ou tipos de dados não carregados. Pulando mapeamento de colunas string.")
# [source: 180]
            return

        self.string_columns_map = {}
        df_cols = list(self.df_original.columns)
        rename_map = {}

        string_cols = [col for col, tipo in self.tipos_dados.items()
                       if tipo == DataType.STRING and col in df_cols]

# [source: 180]
        for col in string_cols:
            try:
# [source: 181]
                col_idx = df_cols.index(col)
                if col_idx + 1 < len(df_cols):
                    pts_original_col = df_cols[col_idx + 1]
                    # Verifica se a próxima coluna NÃO é outro critério conhecido
                    # (uma heurística, poderia ser melhorada se a config indicasse a pts_col explicitamente)
                    if pts_original_col not in self.pesos.index and pts_original_col not in self.tipos_dados:
                        new_pts_col_name = f"{col}{PTS_STRING_SUFFIX}" # Nome padronizado
# [source: 183]
                        self.string_columns_map[col] = new_pts_col_name # Mapeia string -> nome padronizado pts
                        rename_map[pts_original_col] = new_pts_col_name
                        logging.info(f"Coluna string '{col}' mapeada para '{new_pts_col_name}' (original: '{pts_original_col}')")
                    else:
                        logging.warning(f"Coluna '{pts_original_col}' após '{col}' parece ser um critério, não uma coluna de pontos. Mapeamento ignorado.")
                else:
# [source: 184]
                    logging.warning(f"Coluna string '{col}' não tem uma coluna seguinte para ser pts_string.")
            except ValueError:
                 # Isso não deve acontecer se col está em string_cols, mas por segurança
                 logging.warning(f"Coluna string '{col}' definida nos tipos não encontrada nas colunas do DataFrame.")
            except Exception as e:
# [source: 185]
                 logging.error(f"Erro inesperado ao mapear coluna string '{col}': {e}")

        # Renomeia as colunas de pontos no DataFrame original *depois* do loop
        if rename_map:
            self.df_original.rename(columns=rename_map, inplace=True)
            logging.info(f"Colunas de pontos renomeadas no DataFrame original: {rename_map}")


    def _extract_bom_neutro(self) -> None:
        """Extrai as linhas BOM e NEUTRO do DataFrame original."""
        if self.df_original is None: return

        try:
            # Assume que a primeira coluna é o identificador/label
            label_col = self.df_original.columns[0]

# [source: 154]
            bom_row = self.df_original[self.df_original[label_col] == ROW_LABEL_BOM]
            neutro_row = self.df_original[self.df_original[label_col] == ROW_LABEL_NEUTRO]

            if not bom_row.empty:
                self.valores_bom = bom_row.iloc[0].copy() # .copy() para evitar SettingWithCopyWarning
                logging.info(f"Valores BOM extraídos: {self.valores_bom.dropna().to_dict()}")
            else:
                logging.warning(f"Linha '{ROW_LABEL_BOM}' não encontrada.")
                self.valores_bom = pd.Series(dtype='object')

            if not neutro_row.empty:
# [source: 155]
                self.valores_neutro = neutro_row.iloc[0].copy()
                logging.info(f"Valores NEUTRO extraídos: {self.valores_neutro.dropna().to_dict()}")
            else:
                logging.warning(f"Linha '{ROW_LABEL_NEUTRO}' não encontrada.")
                self.valores_neutro = pd.Series(dtype='object')

        except IndexError:
            logging.error("Erro ao acessar colunas/linhas para extrair BOM/NEUTRO. Verifique a estrutura do CSV.")
            self.valores_bom = pd.Series(dtype='object')
            self.valores_neutro = pd.Series(dtype='object')
        except Exception as e:
            logging.error(f"Erro inesperado ao extrair BOM/NEUTRO: {e}")
            self.valores_bom = pd.Series(dtype='object')
            self.valores_neutro = pd.Series(dtype='object')


    def _prepare_calculo_df(self) -> None:
        """Cria o DataFrame de cálculo pulando linhas de config/BOM/NEUTRO."""
        if self.df_original is None: return

# [source: 156]
        self.calculo_df = self.df_original.iloc[DATA_START_ROW:].reset_index(drop=True).copy()
        logging.info(f"DataFrame para cálculo criado com {len(self.calculo_df)} linhas.")


    def _convert_data_types(self) -> None:
        """Converte os tipos de dados no DataFrame de cálculo e nos valores BOM/NEUTRO."""
        if self.calculo_df is None or self.tipos_dados is None or self.valores_bom is None or self.valores_neutro is None:
            logging.warning("Dados insuficientes para conversão de tipos.")
            return

        logging.info("Iniciando conversão de tipos de dados...")
        for col, tipo in self.tipos_dados.items():
            if col not in self.calculo_df.columns:
                 # Colunas de pontos de string (renomeadas) não estarão aqui, o que é esperado.
                 # Elas também não estão em tipos_dados (apenas a string original está).
# [source: 157]
                 if col not in self.string_columns_map: # Se não for uma string mapeada, avisa
                     logging.warning(f"Coluna '{col}' definida nos tipos mas não encontrada nos dados para cálculo.")
                 continue

            bom_val = self.valores_bom.get(col)
            neutro_val = self.valores_neutro.get(col)

            try:
                if tipo == DataType.NUMBER:
                    # Converte BOM/NEUTRO para float, usa NaN se falhar
# [source: 158]
                    self.valores_bom[col] = pd.to_numeric(bom_val, errors='coerce')
                    self.valores_neutro[col] = pd.to_numeric(neutro_val, errors='coerce')
                    # Converte coluna de dados
                    self.calculo_df[col] = pd.to_numeric(self.calculo_df[col], errors='coerce')
                    logging.debug(f"Coluna '{col}' convertida para numérica.")

                elif tipo == DataType.BOOLEAN:
                     # --- CORREÇÃO BOOLEAN ---
                     # Converte BOM/NEUTRO: 'TRUE'/'True'/'true'/1 -> True, outros/NaN -> False
# [source: 159]
                     self.valores_bom[col] = str(bom_val).strip().upper() in ('TRUE', '1')
                     self.valores_neutro[col] = str(neutro_val).strip().upper() in ('TRUE', '1')

                     # Converte coluna de dados com mapeamento robusto
                     bool_map = {'TRUE': True, 'FALSE': False, '1': True, '0': False}
# [source: 160]
                     self.calculo_df[col] = self.calculo_df[col].astype(str).str.strip().str.upper()
                     self.calculo_df[col] = self.calculo_df[col].map(bool_map).fillna(False).astype(bool)
                     logging.debug(f"Coluna '{col}' convertida para booleana.")

                elif tipo == DataType.STRING:
                    # Apenas garante que a coluna de dados seja string (BOM/NEUTRO permanecem como lidos)
# [source: 161]
                    self.calculo_df[col] = self.calculo_df[col].astype(str).str.strip()
                    # Mantém BOM/NEUTRO como strings originais para busca no mapeamento depois
                    self.valores_bom[col] = str(bom_val).strip() if pd.notna(bom_val) else None
                    self.valores_neutro[col] = str(neutro_val).strip() if pd.notna(neutro_val) else None
                    logging.debug(f"Coluna '{col}' mantida/convertida para string.")

                # Tipo PTS_STRING não precisa de conversão aqui, seus valores serão lidos depois.

            except Exception as e:
                 logging.error(f"Erro ao converter tipo para coluna '{col}' (Tipo: {tipo.value}): {e}")
                 # Define a coluna como NaN ou tipo padrão para evitar erros posteriores
                 if tipo == DataType.NUMBER:
                      self.calculo_df[col] = np.nan
                 elif tipo == DataType.BOOLEAN:
                      self.calculo_df[col] = False
                 else:
                      self.calculo_df[col] = ''

        logging.info("Conversão de tipos de dados concluída.")


    def calcular_pontuacoes(self) -> None:
        """
        Calcula as pontuações para cada critério e a pontuação total.
        """
# [source: 192]
        if not self._check_data_ready():
            logging.error("Pré-requisitos para cálculo de pontuações não atendidos.")
            return

        logging.info("Iniciando cálculo de pontuações...")
        if self.calculo_df is None or self.pesos is None: return # Type checking

        # Limpa pontuações de cálculos anteriores
        cols_to_drop = [c for c in self.calculo_df.columns if c.endswith(PONTUACAO_SUFFIX) or c == 'Pontuacao_Total']
        self.calculo_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

        # Pré-calcula mapeamentos de string para otimizar
        self._build_string_to_points_maps()

# [source: 193]
        for col in self.pesos.index:
            self._process_column_score(col)

        # Calcula pontuação total
        self._calculate_total_score()

        logging.info("Cálculo de pontuações concluído.")


    def _check_data_ready(self) -> bool:
        """Verifica se os dados essenciais para cálculo estão carregados."""
        if self.calculo_df is None:
            logging.error("DataFrame de cálculo não está pronto.")
            return False
        if self.tipos_dados is None:
            logging.error("Tipos de dados não foram definidos.")
            return False
        if self.pesos is None:
            logging.error("Pesos não foram definidos.")
            return False
        if self.valores_bom is None or self.valores_neutro is None:
            logging.error("Valores BOM/NEUTRO não foram definidos.")
            return False
        return True

    def _build_string_to_points_maps(self) -> None:
        """
        Constrói os dicionários de mapeamento (string -> ponto base) para todas
        as colunas de string mapeadas, lendo da coluna de pontos correspondente
        no DataFrame *original*.
        """
        if self.df_original is None:
            logging.warning("DataFrame original não disponível para construir mapas de string.")
            return

        self._string_to_points_base_map = {}
        dados_originais = self.df_original.iloc[DATA_START_ROW:].copy()

        for col_str, pts_col_name in self.string_columns_map.items():
            if pts_col_name not in dados_originais.columns:
                logging.warning(f"Coluna de pontos '{pts_col_name}' para '{col_str}' não encontrada no DataFrame original. Mapeamento não criado.")
                continue
            if col_str not in dados_originais.columns:
                 logging.warning(f"Coluna string original '{col_str}' não encontrada no DataFrame original. Mapeamento não criado.")
                 continue

            try:
                # Converte a coluna de pontos para numérico
                dados_originais[pts_col_name] = pd.to_numeric(dados_originais[pts_col_name], errors='coerce')

                # Remove linhas onde a string original OU o ponto convertido são NaN/None
                map_data = dados_originais.dropna(subset=[col_str, pts_col_name])

                # Cria o dicionário de mapeamento (String -> Ponto Numérico Base)
                # Limpa espaços extras das chaves (strings)
                mapping = pd.Series(map_data[pts_col_name].values,
                                    index=map_data[col_str].astype(str).str.strip()).to_dict()

                if not mapping:
                    logging.warning(f"Mapeamento de pontuação para '{col_str}' ficou vazio após limpeza.")
                else:
                    self._string_to_points_base_map[col_str] = mapping
                    logging.info(f"Mapeamento base criado para '{col_str}' com {len(mapping)} entradas.")

            except Exception as e:
                logging.error(f"Erro ao criar mapa de pontos base para '{col_str}': {e}")


    def _process_column_score(self, col: str) -> None:
        """Processa o cálculo da pontuação para uma única coluna."""
        if self.calculo_df is None or self.pesos is None or self.tipos_dados is None \
           or self.valores_bom is None or self.valores_neutro is None:
            return # Já verificado antes, mas seguro

        # Verifica se a coluna de peso existe nos dados de cálculo ou é uma string mapeada
        is_string_col = col in self.string_columns_map
        if col not in self.calculo_df.columns and not is_string_col:
# [source: 194]
            logging.warning(f"Coluna de peso '{col}' não encontrada no DataFrame de cálculo e não é string mapeada. Pulando.")
            return

        tipo = self.tipos_dados.get(col)
        if tipo is None:
            logging.warning(f"Tipo de dado não definido para a coluna de peso '{col}'. Pulando.")
            return

        logging.info(f"Processando coluna: '{col}' (Tipo: {tipo.value})")
        pont_col_name = f"{col}{PONTUACAO_SUFFIX}"
        peso = self.pesos.get(col, 0.0)
        if peso == 0:
            logging.warning(f"Peso para '{col}' é 0. A pontuação será 0.")
            self.calculo_df[pont_col_name] = 0.0
            return

        try:
            if tipo in [DataType.NUMBER, DataType.BOOLEAN]:
                proporcionalidade = self.proporcionalidade.get(col) if self.proporcionalidade else None
                valor_bom = self.valores_bom.get(col)
                valor_neutro = self.valores_neutro.get(col)

                # Verifica se temos tudo para calcular
                if proporcionalidade is None:
# [source: 201]
                    logging.warning(f"Proporcionalidade não definida para '{col}'. Pontuação será 0.")
                    self.calculo_df[pont_col_name] = 0.0
                    return
                if pd.isna(valor_bom) or pd.isna(valor_neutro):
# [source: 199]
                    logging.warning(f"Valores BOM ({valor_bom}) ou NEUTRO ({valor_neutro}) ausentes/inválidos para '{col}'. Pontuação será 0.")
                    self.calculo_df[pont_col_name] = 0.0
                    return

                logging.debug(f"  Tipo: {tipo.value}, Prop: {proporcionalidade.value}, BOM: {valor_bom}, NEUTRO: {valor_neutro}, Peso: {peso}")
                self.calculo_df[pont_col_name] = self.calculo_df.apply(
                    lambda row: self._calcular_pontuacao_numerica_ou_booleana(
                        row.get(col), valor_bom, valor_neutro, peso, proporcionalidade
                    ) if col in row else 0.0,
                    axis=1
                )

            elif tipo == DataType.STRING:
                 # A coluna original (col) DEVE existir no calculo_df para buscarmos o valor
                 if col not in self.calculo_df.columns:
# [source: 206]
                     logging.error(f"Erro Interno: Coluna string '{col}' não encontrada em calculo_df para aplicar pontuação.")
                     self.calculo_df[pont_col_name] = 0.0
                     return

                 # O mapeamento deve ter sido pré-calculado
                 mapping = self._string_to_points_base_map.get(col)
                 if not mapping:
                     logging.warning(f"Mapeamento de pontos base para string '{col}' não disponível. Pontuação será 0.")
                     self.calculo_df[pont_col_name] = 0.0
                     return

                 logging.debug(f"  Tipo: {tipo.value}, Peso: {peso}. Usando mapeamento pré-calculado.")
                 # Passa o mapeamento para a função de cálculo
                 self.calculo_df[pont_col_name] = self.calculo_df[col].apply(
                     lambda val: self._calcular_pontuacao_string(val, peso, mapping)
                 )

            else:
                 logging.warning(f"Tipo de dado não suportado para cálculo direto: '{tipo.value}' para coluna '{col}'.")
                 self.calculo_df[pont_col_name] = 0.0

            # Imprime resultado parcial se a coluna foi criada
            if pont_col_name in self.calculo_df:
                 logging.debug(f"  Pontuações calculadas para '{col}'. Exemplo (Top 5):")
                 logging.debug(f"\n{self.calculo_df[[col, pont_col_name]].head().to_string(index=False)}")

        except KeyError as e:
# [source: 210]
            logging.error(f"Erro de Chave ao processar '{col}': {e}. Verifique nomes nas configs e CSV.")
            if pont_col_name not in self.calculo_df.columns: self.calculo_df[pont_col_name] = 0.0
        except Exception as e:
# [source: 211]
            logging.error(f"Erro inesperado ao processar coluna '{col}': {type(e).__name__} - {str(e)}")
            # import traceback # Descomente para debug detalhado
            # traceback.print_exc()
            if pont_col_name not in self.calculo_df.columns: self.calculo_df[pont_col_name] = 0.0


    def _calculate_total_score(self) -> None:
        """Soma as colunas de pontuação individuais para obter a pontuação total."""
        if self.calculo_df is None: return

        colunas_pontuacao = [c for c in self.calculo_df.columns if c.endswith(PONTUACAO_SUFFIX)]
        if not colunas_pontuacao:
# [source: 212]
            logging.warning("Nenhuma coluna de pontuação individual foi gerada. Pontuação total não calculada.")
            self.calculo_df['Pontuacao_Total'] = 0.0
        else:
            logging.info(f"Calculando Pontuação Total usando: {colunas_pontuacao}")
            # Garante que a soma trate NaNs como 0
            self.calculo_df['Pontuacao_Total'] = self.calculo_df[colunas_pontuacao].sum(axis=1, skipna=True)

        # Mostra resumo final
        if 'Pontuacao_Total' in self.calculo_df:
            logging.info("Resumo final das pontuações (Top 5):")
            modelo_col = self.calculo_df.columns[0] # Assume primeira coluna como identificador
            cols_to_show = [modelo_col] + colunas_pontuacao + ['Pontuacao_Total']
            valid_cols_to_show = [c for c in cols_to_show if c in self.calculo_df.columns]
            logging.info(f"\n{self.calculo_df[valid_cols_to_show].head().to_string(index=False)}")


    def _calcular_pontuacao_numerica_ou_booleana(self, valor: Any, bom: Any, neutro: Any, peso: float,
                                                 proporcionalidade: Proportionality) -> float:
        """
        Calcula a pontuação para um valor numérico ou booleano individual.

        Args:
            valor: O valor do item para o critério.
            bom: O valor de referência 'BOM'.
            neutro: O valor de referência 'NEUTRO'.
            peso: O peso do critério.
            proporcionalidade: O tipo de proporcionalidade (Proportionality.PROPORTIONAL ou Proportionality.I_PROPORTIONAL).

        Returns:
            A pontuação calculada (float).
        """
# [source: 162]
        # 1. Trata o caso booleano primeiro
        if isinstance(valor, (bool, np.bool_)):
            # Caso especial: BOM e NEUTRO são iguais (ambos True ou ambos False)
            if bom == neutro:
                return peso if valor == bom else 0.0 # Pontua se igual ao único alvo, senão 0
            # Caso normal: BOM e NEUTRO são diferentes
            else:
                if valor == bom:
                    return peso  # Acertou o BOM
                elif valor == neutro:
                    return 0.0   # Acertou o NEUTRO
                else:
                    # Não é nem BOM nem NEUTRO (deve ser o oposto do BOM e do NEUTRO)
                    return -peso # Penalidade máxima

        # 2. Lógica numérica (assume que os tipos já foram convertidos para float/NaN)
# [source: 166]
        # Verifica se algum valor essencial é NaN
        if pd.isna(valor) or pd.isna(bom) or pd.isna(neutro) or pd.isna(peso):
            return 0.0 # Não pontua se faltar informação

        # Converte para float por segurança (já deveriam ser, mas garante)
        try:
            valor_f = float(valor)
            bom_f = float(bom)
            neutro_f = float(neutro)
        except (ValueError, TypeError):
            logging.warning(f"Falha na conversão para float em cálculo - Valor: {valor}, Bom: {bom}, Neutro: {neutro}. Retornando 0.")
            return 0.0

        # Se BOM == NEUTRO para números, pontuação é 0 (não há faixa de variação)
# [source: 167]
        if bom_f == neutro_f:
            # Poderia dar pontuação máxima se valor == bom_f, mas manteremos 0 por simplicidade.
            # return peso if valor_f == bom_f else 0.0
            return 0.0

        # Lógica de proporcionalidade
        diff_bom_neutro = bom_f - neutro_f
        # Evita divisão por zero implícita e define denominador seguro para penalidade
        denominador_neutro_seguro = abs(neutro_f) if neutro_f != 0 else 1.0

        try:
            if proporcionalidade == Proportionality.PROPORTIONAL:
                # Caso crescente (maior é melhor)
                if diff_bom_neutro > 0:
                    if valor_f >= bom_f: return peso
                    if valor_f >= neutro_f: return ((valor_f - neutro_f) / diff_bom_neutro) * peso
                    # Abaixo do neutro (penalidade proporcional à distância)
                    return -((abs(neutro_f - valor_f)) / denominador_neutro_seguro) * peso
                # Caso decrescente (menor é melhor) - Incomum para 'proportional'
                else: # diff_bom_neutro < 0
                    if valor_f <= bom_f: return peso
                    if valor_f <= neutro_f: return ((valor_f - bom_f) / diff_bom_neutro) * peso # Invertido
                    # Acima do neutro (penalidade)
                    return -((abs(valor_f - neutro_f)) / denominador_neutro_seguro) * peso

            elif proporcionalidade == Proportionality.I_PROPORTIONAL:
                # Caso decrescente (menor é melhor)
                if diff_bom_neutro < 0: # bom < neutro
                    if valor_f <= bom_f: return peso
                    if valor_f <= neutro_f: return ((neutro_f - valor_f) / (neutro_f - bom_f)) * peso
                    # Acima do neutro (penalidade)
                    return -((abs(valor_f - neutro_f)) / denominador_neutro_seguro) * peso
                # Caso crescente (maior é melhor) - Incomum para 'i_proportional'
                else: # diff_bom_neutro > 0
                    if valor_f >= bom_f: return peso
                    if valor_f >= neutro_f: return ((bom_f - valor_f) / diff_bom_neutro) * peso # Invertido
                     # Abaixo do neutro (penalidade)
                    return -((abs(neutro_f - valor_f)) / denominador_neutro_seguro) * peso

            else:
                # Caso tipo de proporcionalidade não seja reconhecido (não deve acontecer)
                logging.warning(f"Tipo de proporcionalidade desconhecido: {proporcionalidade}")
                return 0.0

        except ZeroDivisionError:
             # Esta exceção não deveria ocorrer devido à checagem bom_f == neutro_f
             logging.error("Erro inesperado de divisão por zero no cálculo numérico.")
             return 0.0


    def _calcular_pontuacao_string(self, valor: str, peso: float, mapping: Dict[str, float]) -> float:
        """
        Calcula a pontuação para um valor de string usando um mapa pré-calculado.

        Args:
            valor: O valor string do item.
            peso: O peso do critério string.
            mapping: O dicionário que mapeia strings limpas para seus pontos base (float).

        Returns:
            A pontuação calculada (float).
        """
# [source: 185]
        if not isinstance(valor, str):
            valor = str(valor) # Garante que é string
        valor_clean = valor.strip()

        # Busca o ponto BASE no mapeamento (retorna 0 se não encontrado)
# [source: 190]
        pontuacao_base = mapping.get(valor_clean, 0.0)

        # Multiplica o ponto base pelo peso da coluna string
        resultado = pontuacao_base * peso

        # logging.debug(f"  String: Val='{valor_clean}', BasePts={pontuacao_base:.2f}, Peso={peso:.2f}, Result={resultado:.2f}")
        return resultado


    # --- Métodos de Exibição e Gráficos ---

    def mostrar_pontuacoes(self, apenas_pontuacoes: bool = True) -> None:
      """
      Exibe o DataFrame com as pontuações calculadas.

      Args:
          apenas_pontuacoes: Se True, mostra apenas a coluna identificadora e
                             as colunas de pontuação (_pontuacao e Total).
                             Se False, mostra todas as colunas do calculo_df.
      """
# [source: 215]
      if self.calculo_df is None:
           logging.warning("DataFrame de cálculo não disponível para exibição.")
           return

      try:
           # Tenta usar display se em ambiente interativo (Jupyter, IPython)
           from IPython.display import display
# [source: 216]
      except ImportError:
           display = print # Fallback para print normal

      df_to_show = self.calculo_df

      if apenas_pontuacoes:
          # Assume que a primeira coluna é o identificador
          modelo_col = self.calculo_df.columns[0]
# [source: 217]
          cols_pontuacao = [c for c in self.calculo_df.columns if c.endswith(PONTUACAO_SUFFIX)]
          cols_total = ['Pontuacao_Total'] if 'Pontuacao_Total' in self.calculo_df.columns else []
          valid_cols = [modelo_col] + cols_pontuacao + cols_total
          # Garante que todas as colunas selecionadas realmente existem
          cols_existentes = [c for c in valid_cols if c in self.calculo_df.columns]
          if not cols_existentes:
              logging.warning("Nenhuma coluna de pontuação encontrada para exibir.")
              return
          df_to_show = self.calculo_df[cols_existentes]

      logging.info("Exibindo DataFrame de Pontuações:")
      display(df_to_show)


    def gerar_graficos(self) -> None:
        """Gera e exibe os gráficos de análise."""
# [source: 218]
        logging.info("\n--- Gerando Gráficos ---")
        if self.calculo_df is None or self.tipos_dados is None:
            logging.error("DataFrame de cálculo ou tipos de dados não disponíveis para gerar gráficos.")
            return
        if self.valores_bom is None or self.valores_neutro is None:
             logging.error("Valores BOM/NEUTRO não disponíveis para gerar gráficos.")
             return

        try:
            self._gerar_grafico_barras_total()
            self._gerar_graficos_numericos()
            self._gerar_graficos_strings()
            self._gerar_graficos_booleanos()
            logging.info("--- Geração de Gráficos Concluída ---")
        except Exception as e:
            logging.error(f"Erro inesperado ao gerar gráficos: {e}")
            # import traceback # Descomente para debug
            # traceback.print_exc()


    def _configure_plot_layout(self, fig: go.Figure, title: str, xaxis_title: Optional[str] = None,
                                yaxis_title: Optional[str] = None, legend_title: Optional[str] = None,
                                y_category_order: Optional[str] = None) -> None:
        """Aplica configurações de layout comuns aos gráficos plotly."""
        layout_options = {
            'title': title,
            'plot_bgcolor': 'white',
            'paper_bgcolor': 'white',
            'xaxis_title': xaxis_title,
            'yaxis_title': yaxis_title,
            'legend_title_text': legend_title
        }
        if y_category_order:
            layout_options['yaxis'] = {'categoryorder': y_category_order}

        fig.update_layout(**layout_options)

    def _adicionar_linha_referencia(self, fig: go.Figure, value: float, text: str, orientation: str = 'v',
                                    y_pos: Optional[float] = None, x_pos: Optional[float] = None,
                                    color: str = 'black', dash: str = 'dash') -> None:
        """Adiciona uma linha de referência vertical ou horizontal com anotação."""
        if orientation == 'v':
            fig.add_vline(x=value, line_dash=dash, line_color=color)
            x_anchor = value
            y_anchor = y_pos if y_pos is not None else fig.layout.yaxis.range[1] # Default to top
            ax = 0
            ay = -40 # Offset da anotação
        elif orientation == 'h':
            fig.add_hline(y=value, line_dash=dash, line_color=color)
            y_anchor = value
            x_anchor = x_pos if x_pos is not None else fig.layout.xaxis.range[1] # Default to right
            ax = -40 # Offset da anotação
            ay = 0
        else:
            logging.warning(f"Orientação de linha de referência inválida: {orientation}")
            return

        fig.add_annotation(
            x=x_anchor, y=y_anchor,
            text=text,
            showarrow=True, arrowhead=1, arrowwidth=1, arrowcolor="#636363",
            font=dict(size=12, color="#000000"), align="center",
            ax=ax, ay=ay, # Controla a posição relativa da seta/texto
            bordercolor="#c7c7c7", borderwidth=1, borderpad=4,
            bgcolor='rgba(255,255,255,0.8)', # Fundo semi-transparente
        )


    def _gerar_grafico_barras_total(self) -> None:
        """Gera o gráfico de barras para a pontuação total."""
        logging.info("Gerando gráfico de barras da Pontuação Total...")
        if self.calculo_df is None or 'Pontuacao_Total' not in self.calculo_df.columns:
# [source: 236]
            logging.warning("Coluna 'Pontuacao_Total' não encontrada. Pulando gráfico de barras.")
            return
        if self.pesos is None:
            logging.warning("Pesos não definidos. Pontuação máxima teórica não pode ser calculada.")
            max_score_teorico = self.calculo_df['Pontuacao_Total'].max() # Usa o máximo atingido como fallback
            max_score_text = f"Máx Atingido ({max_score_teorico:.2f})"
        else:
# [source: 238]
            max_score_teorico = self.pesos.sum()
            max_score_text = f"Desejável ({max_score_teorico:.2f})"
            logging.info(f"  Pontuação máxima teórica (soma pesos): {max_score_teorico:.2f}")

# [source: 239]
        modelo_col = self.calculo_df.columns[0]
        df_analise = self.calculo_df[[modelo_col, 'Pontuacao_Total']].copy()
        df_analise = df_analise[~df_analise[modelo_col].isin([ROW_LABEL_BOM, ROW_LABEL_NEUTRO])]
        df_analise.dropna(subset=['Pontuacao_Total'], inplace=True)

        if df_analise.empty:
# [source: 240]
            logging.warning("Nenhum dado válido para plotar no gráfico de barras total.")
            return

        df_analise['Color_Category'] = np.where(df_analise['Pontuacao_Total'] >= 0, 'Positiva', 'Negativa')
        color_map = {'Positiva': 'lightgreen', 'Negativa': 'palevioletred'}
# [source: 241]
        df_analise_sorted = df_analise.sort_values('Pontuacao_Total', ascending=False) # Melhor no topo

        # Identifica o melhor
        top_product = df_analise_sorted.iloc[0] if not df_analise_sorted.empty else None
# [source: 242]
        if top_product is not None:
            logging.info(f"  Melhor produto: {top_product[modelo_col]} (Score: {top_product['Pontuacao_Total']:.2f})")

        fig = px.bar(df_analise_sorted,
                     x='Pontuacao_Total', y=modelo_col, orientation='h',
                     color='Color_Category', color_discrete_map=color_map,
                     hover_name=modelo_col, hover_data={'Pontuacao_Total': ':.2f'},
                     title='Pontuação Total dos Modelos')

        self._configure_plot_layout(fig, title='Pontuação Total dos Modelos',
                                    xaxis_title='Pontuação Total', yaxis_title='Modelo',
                                    legend_title='Resultado', y_category_order='total ascending')

        # Linhas de referência e anotações
        num_items = len(df_analise_sorted)
        self._adicionar_linha_referencia(fig, 0.0, "Mínimo Aceitável (0.0)", orientation='v', y_pos=num_items - 0.5)
        if max_score_teorico is not None and not pd.isna(max_score_teorico):
            self._adicionar_linha_referencia(fig, max_score_teorico, max_score_text, orientation='v', y_pos=num_items - 1) # Y um pouco mais baixo

        # Anotação para o melhor produto
        if top_product is not None:
            fig.add_annotation(
                x=top_product['Pontuacao_Total'], y=top_product[modelo_col],
                text="🏆 Melhor", showarrow=False,
                font=dict(color="#00008B", size=12),
                bgcolor="rgba(255,255,255,0.7)", bordercolor="black", borderwidth=1, borderpad=4,
                xshift=10, yshift=-15 # Ajuste fino da posição do texto
            )

        fig.show()


    def _gerar_graficos_numericos(self) -> None:
        """Gera gráficos de barras individuais para colunas numéricas."""
        if self.calculo_df is None or self.tipos_dados is None or self.proporcionalidade is None \
            or self.valores_bom is None or self.valores_neutro is None: return

        numeric_cols = [col for col, tipo in self.tipos_dados.items()
                        if tipo == DataType.NUMBER and col in self.calculo_df.columns]
# [source: 221]
        if not numeric_cols:
            logging.info("Nenhuma coluna numérica encontrada para gerar gráficos individuais.")
            return

        modelo_col = self.calculo_df.columns[0]

        for col_num in numeric_cols:
            logging.info(f"Gerando gráfico para coluna numérica: '{col_num}'")

            df_num = self.calculo_df[[modelo_col, col_num]].copy()
            df_num = df_num[~df_num[modelo_col].isin([ROW_LABEL_BOM, ROW_LABEL_NEUTRO])]
            df_num.dropna(subset=[col_num], inplace=True)

            if df_num.empty:
# [source: 222]
                logging.warning(f"Nenhum dado válido para plotar no gráfico de '{col_num}'.")
                continue

            valor_bom_num = pd.to_numeric(self.valores_bom.get(col_num), errors='coerce')
            valor_neutro_num = pd.to_numeric(self.valores_neutro.get(col_num), errors='coerce')
            prop = self.proporcionalidade.get(col_num)
# [source: 224]
            inversamente_proporcional = prop == Proportionality.I_PROPORTIONAL
            titulo = f"{col_num} {'(Menor é Melhor)' if inversamente_proporcional else '(Maior é Melhor)'}"

            # Classifica como positivo/negativo/neutro
            df_num['Color_Category'] = df_num[col_num].apply(
                 lambda x: self._classificar_valor_numerico(x, valor_bom_num, valor_neutro_num, inversamente_proporcional)
            )
            color_map = {'Bom': 'lightgreen', 'Ruim': 'palevioletred', 'Neutro': 'lightgray'}
# [source: 226]
            df_num_sorted = df_num.sort_values(col_num, ascending=inversamente_proporcional) # Ordena pelo valor

            fig = px.bar(df_num_sorted, x=col_num, y=modelo_col, orientation='h',
                         color='Color_Category', color_discrete_map=color_map,
                         hover_name=modelo_col, hover_data={col_num: ':.2f'}, title=titulo)

            self._configure_plot_layout(fig, title=titulo, xaxis_title=col_num, yaxis_title='Modelo',
                                        legend_title='Classificação', y_category_order='total ascending')

            # Linhas de referência BOM/NEUTRO
            num_items = len(df_num_sorted)
            if pd.notna(valor_neutro_num):
                self._adicionar_linha_referencia(fig, valor_neutro_num, f"NEUTRO ({valor_neutro_num:.2f})", 'v', num_items - 0.5)
            if pd.notna(valor_bom_num):
                 self._adicionar_linha_referencia(fig, valor_bom_num, f"BOM ({valor_bom_num:.2f})", 'v', num_items - 1)

            fig.show()

    def _classificar_valor_numerico(self, valor: float, bom: float, neutro: float, invertido: bool) -> str:
        """Classifica um valor numérico como Bom, Ruim ou Neutro em relação a BOM/NEUTRO."""
        if pd.isna(valor) or pd.isna(neutro):
            return 'Neutro' # Se não há referência neutra ou valor, é neutro

        if bom == neutro: # Se BOM e NEUTRO são iguais
             return 'Bom' if valor == bom else ('Ruim' if (invertido and valor > neutro) or (not invertido and valor < neutro) else 'Neutro')

        if invertido: # Menor é melhor
            if pd.notna(bom) and valor <= bom: return 'Bom'
            if valor <= neutro: return 'Bom' # Mesmo que não atinja o 'bom' ótimo, abaixo do neutro é bom
            return 'Ruim' # Acima do neutro é ruim
        else: # Maior é melhor
            if pd.notna(bom) and valor >= bom: return 'Bom'
            if valor >= neutro: return 'Bom' # Mesmo que não atinja o 'bom' ótimo, acima do neutro é bom
            return 'Ruim' # Abaixo do neutro é ruim


    def _gerar_graficos_strings(self) -> None:
        """Gera gráficos de dispersão para colunas string usando pontos base."""
        if self.calculo_df is None or self.tipos_dados is None or not self._string_to_points_base_map \
           or self.valores_bom is None or self.valores_neutro is None:
            logging.info("Pré-requisitos para gráficos de string não atendidos ou nenhum mapa de string criado.")
            return

        modelo_col = self.calculo_df.columns[0]

        for col_string, mapping in self._string_to_points_base_map.items():
             if col_string not in self.calculo_df.columns: continue # Segurança

             logging.info(f"Gerando gráfico para string: '{col_string}'")

             df_string = self.calculo_df[[modelo_col, col_string]].copy()
             df_string = df_string[~df_string[modelo_col].isin([ROW_LABEL_BOM, ROW_LABEL_NEUTRO])]
             df_string.dropna(subset=[col_string], inplace=True)

             if df_string.empty:
                 logging.warning(f"Nenhum dado para plotar no gráfico de string para '{col_string}'.")
                 continue

             # Mapeia para obter o PONTO BASE para o gráfico
             df_string['ValorBase'] = df_string[col_string].astype(str).str.strip().map(mapping).fillna(0)
# [source: 266]
             df_string = df_string.sort_values('ValorBase', ascending=False)
             df_string['Posicao'] = range(len(df_string)) # Posição X baseada na ordenação

             fig = px.scatter(df_string, x='Posicao', y='ValorBase',
                              color=col_string, # Cor baseada na categoria string
                              hover_name=modelo_col, # O que aparece ao passar o mouse
                              size_max=15, marker=dict(size=10),
                              hover_data={'Posicao': False, 'ValorBase': ':.2f', col_string: True},
                              labels={'ValorBase': 'Pontuação Base', col_string: col_string.capitalize()})

             # Adiciona linhas de referência BOM/NEUTRO com base nos pontos BASE
             valor_bom_str = self.valores_bom.get(col_string)
             valor_neutro_str = self.valores_neutro.get(col_string)
             valor_base_bom = mapping.get(str(valor_bom_str).strip()) if valor_bom_str is not None else None
             valor_base_neutro = mapping.get(str(valor_neutro_str).strip()) if valor_neutro_str is not None else None

             # Calcula posição x para anotações hline (ex: 90% da largura)
             x_pos_annot = (df_string['Posicao'].max() - df_string['Posicao'].min()) * 0.9 + df_string['Posicao'].min()

             if valor_base_bom is not None:
                  self._adicionar_linha_referencia(fig, valor_base_bom, f"BOM: '{valor_bom_str}' ({valor_base_bom:.2f})",
                                                   'h', x_pos=x_pos_annot, color='green')
             if valor_base_neutro is not None:
                  self._adicionar_linha_referencia(fig, valor_base_neutro, f"NEUTRO: '{valor_neutro_str}' ({valor_base_neutro:.2f})",
                                                   'h', x_pos=x_pos_annot, color='orange')


             self._configure_plot_layout(fig, title=f'Comparação de {col_string.capitalize()} (Pontuação Base)',
                                        xaxis_title='Modelos Ordenados por Pontuação Base',
                                        yaxis_title='Pontuação Base (Definida no CSV)',
                                        legend_title=col_string.capitalize())

             fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
             fig.update_yaxes(showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='lightgray')
             fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=-0.3, xanchor='center', x=0.5))
             fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey'), opacity=0.8))

             fig.show()


    def _gerar_graficos_booleanos(self) -> None:
        """Gera gráficos de dispersão (Sim/Não) para colunas booleanas."""
        if self.calculo_df is None or self.tipos_dados is None or self.valores_bom is None or self.valores_neutro is None: return

        bool_cols = [col for col, tipo in self.tipos_dados.items()
                     if tipo == DataType.BOOLEAN and col in self.calculo_df.columns]
# [source: 275]
        if not bool_cols:
            logging.info("Nenhuma coluna booleana encontrada para gerar gráficos individuais.")
            return

        modelo_col = self.calculo_df.columns[0]

        for col_bool in bool_cols:
            logging.info(f"Gerando gráfico para booleano: '{col_bool}'")

            df_bool = self.calculo_df[~self.calculo_df[modelo_col].isin([ROW_LABEL_BOM, ROW_LABEL_NEUTRO])][[modelo_col, col_bool]].copy()
            df_bool.dropna(subset=[col_bool], inplace=True) # Remove NaN se houver

            if df_bool.empty:
                 logging.warning(f"Nenhum dado válido para plotar no gráfico booleano de '{col_bool}'.")
                 continue

            df_bool['Valor'] = df_bool[col_bool].map({True: 1, False: 0})
            df_bool['Tamanho'] = 15 # Tamanho fixo para os pontos

            fig = px.scatter(df_bool.sort_values(modelo_col), # Ordena por nome para consistência
                             x=modelo_col, y='Valor',
                             color=col_bool,
                             color_discrete_map={True: '#4CAF50', False: '#F44336'}, # Verde/Vermelho
                             size='Tamanho', size_max=20,
                             hover_name=modelo_col,
                             hover_data={'Valor': False, 'Tamanho': False, col_bool: True})

            # Atualiza legenda e eixo Y
            fig.for_each_trace(lambda t: t.update(name='Sim' if t.name == 'True' else 'Não'))
            fig.update_yaxes(range=[-0.5, 1.5], tickvals=[0, 1], ticktext=['Não', 'Sim'], showgrid=False)

            # Linhas de referência BOM/NEUTRO (convertidos para 0/1)
            valor_bom_bool = 1 if self.valores_bom.get(col_bool, False) else 0
            valor_neutro_bool = 1 if self.valores_neutro.get(col_bool, False) else 0

            # Calcula posição x para anotações hline
            x_pos_annot = len(df_bool[modelo_col].unique()) - 0.5 # Perto da borda direita

            self._adicionar_linha_referencia(fig, valor_neutro_bool, f"NEUTRO ({'Sim' if valor_neutro_bool else 'Não'})",
                                             'h', x_pos=x_pos_annot, color='orange')
            if valor_bom_bool != valor_neutro_bool: # Só adiciona BOM se for diferente
                self._adicionar_linha_referencia(fig, valor_bom_bool, f"BOM ({'Sim' if valor_bom_bool else 'Não'})",
                                                 'h', x_pos=x_pos_annot, color='green')

            self._configure_plot_layout(fig, title=f'{col_bool.capitalize()} por Modelo',
                                        xaxis_title=None, yaxis_title=None,
                                        legend_title='Valor')
            fig.update_layout(xaxis_tickangle=-45) # Melhora leitura dos nomes no eixo X

            fig.show()


    # --- Recomendação ---

    def recomendar_produtos(self, top_n: int = 1) -> None:
        """
        Recomenda os 'top_n' melhores produtos com base na pontuação total e
        detalha os pontos fortes e fracos de cada um.

        Args:
            top_n: O número de melhores produtos a serem recomendados.
        """
# [source: 282]
        logging.info(f"\n--- Recomendando Top {top_n} Produto(s) ---")
        if self.calculo_df is None or 'Pontuacao_Total' not in self.calculo_df.columns:
# [source: 283]
            logging.error("Pontuação total não calculada. Não é possível recomendar.")
            return

        modelo_col = self.calculo_df.columns[0]
        produtos = self.calculo_df[~self.calculo_df[modelo_col].isin([ROW_LABEL_BOM, ROW_LABEL_NEUTRO])].copy()
        produtos.dropna(subset=['Pontuacao_Total'], inplace=True)

        if produtos.empty:
# [source: 284]
            logging.warning("Nenhum produto encontrado para recomendação (após excluir BOM/NEUTRO e NaN).")
            return

        # Ordena por pontuação total (maior primeiro)
        recomendados = produtos.sort_values('Pontuacao_Total', ascending=False).head(top_n)

        if recomendados.empty:
            logging.warning("Nenhum produto restante após ordenação e seleção do top N.")
            return

        # Ordena critérios por peso absoluto (maior impacto primeiro) para exibição
        if self.pesos is None or self.tipos_dados is None:
             logging.error("Pesos ou tipos não disponíveis para detalhar recomendação.")
             return

        criterios_ordenados = sorted(
            [(col, abs(peso)) for col, peso in self.pesos.items() if col in self.tipos_dados], # Considera apenas critérios com tipo definido
            key=lambda item: item[1], reverse=True
        )
# [source: 287]

        for idx, produto in recomendados.iterrows():
            print(f"\n{idx+1}. Produto Recomendado: {produto[modelo_col]} (Pontuação Total: {produto['Pontuacao_Total']:.2f})")
            print("-" * (len(produto[modelo_col]) + 35))
            print("  Detalhes da Pontuação por Critério (Ordenado por Peso):")

            for col, peso_abs in criterios_ordenados:
                 # Precisa do peso original (com sinal) para a justificativa
                 peso_original = self.pesos.get(col, 0.0)
                 pont_col = f"{col}{PONTUACAO_SUFFIX}"

                 # Verifica se a coluna de pontuação existe e tem valor válido
                 if pont_col not in produto or pd.isna(produto[pont_col]):
                     # logging.debug(f"  - {col}: Pontuação não disponível ou inválida.")
                     continue # Pula se não houver pontuação para este critério

                 pontuacao = produto[pont_col]
                 valor_atual = produto.get(col, "N/A") # Pega o valor original com segurança
                 tipo = self.tipos_dados.get(col)

                 # Formata a saída para melhor leitura
                 justificativa = f"(Peso: {peso_original:.2f}, Pontos: {pontuacao:.2f})"
                 prefixo = f"  - {col}: "
                 sufixo = ""

                 # Define se é Vantagem/Desvantagem/Neutro baseado na pontuação
                 if pontuacao > 0.01 * abs(peso_original): # Considera vantagem se > 1% do peso
                     sufixo = f"[Vantagem] {justificativa}"
                 elif pontuacao < -0.01 * abs(peso_original): # Considera desvantagem se < -1% do peso
                     sufixo = f"[Desvantagem] {justificativa}"
                 else:
                     sufixo = f"[Neutro] {justificativa}"

                 # Formata o valor atual baseado no tipo
                 if tipo == DataType.STRING:
                     valor_atual_str = f"'{valor_atual}'" if valor_atual != "N/A" else valor_atual
                     print(f"{prefixo}{valor_atual_str} {sufixo}")
                 elif tipo == DataType.BOOLEAN:
                     valor_atual_str = "Sim" if valor_atual is True else ("Não" if valor_atual is False else "N/A")
                     print(f"{prefixo}{valor_atual_str} {sufixo}")
                 elif tipo == DataType.NUMBER:
                     try:
                         valor_atual_str = f"{float(valor_atual):.1f}" if pd.notna(valor_atual) else "N/A"
                     except (ValueError, TypeError):
                          valor_atual_str = str(valor_atual) # Mantém como string se não for número
                     # Adiciona BOM/NEUTRO para contexto
                     valor_bom_num = pd.to_numeric(self.valores_bom.get(col), errors='coerce')
                     valor_neutro_num = pd.to_numeric(self.valores_neutro.get(col), errors='coerce')
                     contexto_num = ""
                     if pd.notna(valor_bom_num) and pd.notna(valor_neutro_num):
                          contexto_num = f" (Bom: {valor_bom_num:.1f}, Neutro: {valor_neutro_num:.1f})"

                     print(f"{prefixo}{valor_atual_str}{contexto_num} {sufixo}")
                 else:
                     # Caso tipo desconhecido ou PTS_STRING (não deve ter pontuação direta)
                     print(f"{prefixo}{valor_atual} [Tipo {tipo.value if tipo else 'Desconhecido'}] {justificativa}")


# --- Exemplo de Uso ---
if __name__ == "__main__":
    # Use um bloco try-except para cada análise ou um loop
    arquivos_para_analisar = ['base_eletronicos_2.csv'] # Adicione outros arquivos aqui

    for arquivo in arquivos_para_analisar:
        print(f"\n{'='*10} Iniciando Análise para: {arquivo} {'='*10}")
        try:
            analyzer = DataAnalyzer(arquivo)
            analyzer.load_and_prepare_data()

            # Verifica se o carregamento foi bem-sucedido antes de continuar
            if analyzer.calculo_df is not None:
                 analyzer.calcular_pontuacoes()
                 analyzer.mostrar_pontuacoes(apenas_pontuacoes=True) # Mostra resumo das pontuações
                 analyzer.gerar_graficos()
                 analyzer.recomendar_produtos(top_n=2) # Recomenda os 2 melhores
            else:
                 logging.error(f"Análise não pode continuar para {arquivo} devido a erros no carregamento/preparação.")

        except FileNotFoundError:
            logging.error(f"Arquivo '{arquivo}' não encontrado.")
        except Exception as e:
            logging.error(f"Ocorreu um erro inesperado durante a análise de '{arquivo}': {e}", exc_info=True) # exc_info=True mostra o traceback
        print(f"{'='*10} Fim da Análise para: {arquivo} {'='*10}")


========== Iniciando Análise para: base_eletronicos_2.csv ==========


,Modelo,Memória RAM_pontuacao,Armazenamento_pontuacao,Tamanho de tela_pontuacao,Bateria mAh_pontuacao,Cor_pontuacao,Bluetooth_pontuacao,Pontuacao_Total
0,Acer,-0.15,0.200000,0.10000,0.10,0.0,0.1,0.350000
1,Lenovo,0.00,0.085714,0.05000,0.00,0.0,0.1,0.235714
2,DELL,0.15,0.085714,0.05000,0.00,0.0,0.0,0.285714
3,Samsung,0.30,0.028571,0.10000,0.10,0.0,0.1,0.628571
4,Positivo,0.30,0.000000,0.00000,0.20,0.0,0.0,0.500000
5,Asus,0.30,0.093333,0.10000,0.15,0.0,0.1,0.743333
6,HP,0.30,0.032381,0.10000,0.05,0.0,0.1,0.582381
7,Apple,0.30,0.200000,0.00000,0.20,0.0,0.1,0.800000
8,LG,-0.15,-0.100000,-0.00625,-0.04,0.0,0.0,-0.296250



8. Produto Recomendado: Apple (Pontuação Total: 0.80)
----------------------------------------
  Detalhes da Pontuação por Critério (Ordenado por Peso):
  - Memória RAM: 16.0 (Bom: 8.0, Neutro: 4.0) [Vantagem] (Peso: 0.30, Pontos: 0.30)
  - Armazenamento: 1000.0 (Bom: 960.0, Neutro: 120.0) [Vantagem] (Peso: 0.20, Pontos: 0.20)
  - Bateria mAh: 6000.0 (Bom: 5000.0, Neutro: 3000.0) [Vantagem] (Peso: 0.20, Pontos: 0.20)
  - Tamanho de tela: 16.0 (Bom: 14.0, Neutro: 16.0) [Neutro] (Peso: 0.10, Pontos: 0.00)
  - Cor: 'Cinza' [Neutro] (Peso: 0.10, Pontos: 0.00)
  - Bluetooth: Sim [Vantagem] (Peso: 0.10, Pontos: 0.10)

6. Produto Recomendado: Asus (Pontuação Total: 0.74)
---------------------------------------
  Detalhes da Pontuação por Critério (Ordenado por Peso):
  - Memória RAM: 12.0 (Bom: 8.0, Neutro: 4.0) [Vantagem] (Peso: 0.30, Pontos: 0.30)
  - Armazenamento: 512.0 (Bom: 960.0, Neutro: 120.0) [Vantagem] (Peso: 0.20, Pontos: 0.09)
  - Bateria mAh: 4500.0 (Bom: 5000.0, Neutro: 3000.0)